# Pronalaženje regulatornih motiva u skupu nukleotidnih sekvenci DNA 

# Sadržaj<a id="par:toc"></a>

1. [Uvod](#par:uvod)
2. [Deterministički algoritmi](#par:det)  
   2.1. [Prva formulacija](#par:prvafor)  
   2.2. [Druga formulacija](#par:drugafor)  
   2.3. [Problem niske medijane](#par:niskamedijana) 
2. [Probabilistički algoritmi](#par:prob)  
   3.1. [Pohlepna pretraga motiva](#par:greedymotifsearch)   
   3.2. [Poboljšana pohlepna pretraga motiva](#par:poboljsanapohlepnapret)   
   3.3. [Probabilistička pretraga motiva](#par:randomizovanapretragamotiva)   
   3.4. [Gibsovo uzorkovanje](#par:gibsovosempliranje)    
   3.3. [EM algoritam](#par:emalgoritam)   
4. [Zaključak](#par:zaključak) 


# Uvod <a id="par:uvod"></a>


Raspored aktivnosti  biljaka, životinja i bakterija u toku dana kontroliše "unutrašnji časovnik" koji se naziva cirkadijalni sat. Na Slici  [1.1] su predstavljeni različiti procesi koji su kontrolisani ovim satom. Postoji tačno vreme kada telo u toku dana ima najnižu temperaturu, kada kreće i prestaje lučenje hormona melatonina i slično.

[1.1]: #fig:bio0

<figure><img src="bioinformatika0.png" width="60%" id="fig:bio0" /><figcaption style="text-align: center;"><b>Slika 1.1</b>: Cirkadijalni ritam</figcaption></figure>

Cirkadijalnim satom upravljaju molekularni mehanizmi, a jedan od njih je regulacija sinteze proteina. S obzirom na to, možemo postaviti pitanje kako ćelije znaju kada da uspore, a kada da ubrzaju proizvodnju odgovarajućih proteina. Geni koji regulišu cirkadijalni ritam organizama se nazivaju cirkadijalni geni. Ovi geni kontrolišu mnoge procese i funkcije koji se ponavljaju u organizumu na svakih $24$ časa, kao što su  spavanje, disanje, lučenje hormona i  koordiniraju ponašanje stotine drugih gena. Fokusiraćemo se na biljke, jer kod njih cirkadijalni sat igra ključnu ulogu u njihovom rastu i razvoju.  Biolozi procenjuju da su preko hiljadu gena kod biljaka cirkadijalni, uključujući gene koji se odnose na fotosintezu, fotoreceptore i cvetanje. Zaključeno je da svaka ćelija prati dan i  noć nezavisno od drugih ćelija i da su tri biljna gena LCY, CCA1 i  TOC1, tačnije proteini koje ovi geni kodiraju odgovorna za upravljanje cirkadijalnim genima.

Unutar $DNK$ su zapisane informacije za sintezu proteina u ćeliji. Svaki segment $DNK$ koji sadrži takvu informaciju predstavlja jedan gen. Pored $DNK$, postoje i različite vrste ribonukleinskih kiselina, odnosno $RNK$. Dok je $DNK$ predstavljen dvosturkim lancem koji čine nukleotidi adenin (A), citozin (C), guanin (G), timin (T), $RNK$ je predstavljen jednostrukim lancem. Umesto timina, kod $RNK$ se pojavljuje nukelotid uracil koji se označava sa U. Da bi nastali proteini neophodno je da se na osnovu dva lanca od $DNK$ konstruiše $RNK$ molekul. Formiranje $RNK$ od $DNK$ predstavlja jednostavno prepisivanje iz oba lanca $DNK$, uz zamenu nukleotida T sa U. Pomenuto prepisivanje nukeotida predstavlja prvi korak sinteze proteina, tj. transkripciju gena. Početak ovog procesa iniciraju posebna jedinjenja, takozvani transkripcioni faktori (kod biljaka to su uglavnom pomenuti LCY, CCA1 i TOC1), koji se vezuju za kratke $DNK$ segmente ispred gena. Ovi kratki segmenti nazivaju se regulatorni motivi. 

Drugi korak jeste prevođenje, odnosno translacija, prepisanog $RNK$ u proteine. Imamo $4$ nukelotida A, C, G, U i njih treba da prevedemo u $20$ mogućih aminokiselina. Genetski kod predstavlja preslikavanje skupa kodona ($3$-grama) u skup  aminokiselina (Slika [1.2]). Kako preslikavamo redom kodone iz $RNK$ u proteine, potrebno je da znamo gde je kraj nekog proteina. Postoje STOP kodoni koji označavaju da iza njih nema više kodona aminokiselina koji kodiraju protein. Ti stop kodoni su UAA, UAG, UGA. 

[1.2]: #fig:genetskikod

<figure><img src="genetskikod.png" width="40%" id="fig:genetskikod" /><figcaption style="text-align: center;"><b>Slika 1.2</b>: Genetski kod </figcaption></figure>

Problem pronalaženja motiva bi bio lakši da su regulatorni motivi potpuno očuvani. Ipak, regulatorni motivi mogu da mutiraju, kao i ostali delovi DNK. Pitanje je kako da nađemo regulatorne motive ako ne znamo kako izgledaju. Kroz ovaj rad će biti prikazano nekoliko algoritama za pronalaženje regulatornih motiva. 


Fokus ovog rada je na elektronskoj lekciji  koja će biti deo šireg elektronskog udžbenika iz bioinformatike. U tekstu su objašnjeni, a u lekciji implementirani  svi algoritmi koji su predstavljeni u knjizi $Bioinformatics$ $Algorithms$: $An$ $Active$
$Learning$ $Approach$ autora Filipa Kompoa ($Phillip$ $Compeau$) i Pavela Pevznera ($Pavel$ $Pevzner$). Navedena knjiga je glavni udžbenik kursa Uvod u bioinformatiku. Rezultat rada je $Jupyter$ sveska sa implementiranim algoritmima za pronalaženje regulatornih motiva koji su pisani u $Python$-u.

# Deterministički algoritmi<a id="par:det"></a>

Za datu kolekciju nukeotidnih sekvenci, želimo da nađemo podsekvence, po jednu u svakoj sekvenci, koje su međusobno slične. Ovaj biološki problem je poznat kao problem pronalaženja motiva. Ovom problemu se može pristupati na različite načine, pa će kroz rad biti prikazano više njegovih formulacija. 

<figure><img src="bioinformatika1.png" width="65%" id="fig:bio1" /><figcaption style="text-align: center;"><b>Slika 2.1</b>: Deset nasumično generisanih nukleotidnih sekvenci takvih da je u svaku sekvencu ugrađen isti šablon </figcaption></figure>

Kako bismo simulirali realnu kolekciju sekvenci u kojoj bismo tražili regulatorne motive, generisaćemo na slučajan način 10 nukleotidnih sekvenci dužine $600$ i u svaku sekvencu ubaciti  šablon AAAAAAAAGGGGGGG  dužine $15$ na proizvoljnu poziciju. Na Slici [2.1] je prikazan segment ove kolekcije. Umesto da svaka sekvenca sadrži isti ubačeni šablon, on se može mutirati na proizvoljno odabranim pozicijama. Primer mutiranja šablona AAAAAAAAGGGGGGG u svakoj  nukelotidnoj sekvenci na $4$ proizvoljno odabrane pozicije možemo videti na Slici [2.2]. Ova kolekcija je poznata pod nazivom $Subtle$ $Motif$ $Search$ kolekcija i u nastavku rada će poslužiti kao $benchmark$ kolekcija za testiranje različitih algoritama za rešavanje problema nalaženja motiva.

[2.1]: #fig:bio1
[2.2]: #fig:bio2

<figure><img src="bioinformatika2.png" width="65%" id="fig:bio2" /><figcaption style="text-align: center;"><b>Slika 2.2</b>: Deset nasumično generisanih nukelotidnih sekvenci takvih da je u svaku sekvencu ugrađen po jedan mutirani šablon</figcaption></figure>

Na osnovu prethodno opisane kolekcije sekvenci sa ugrađenim šablonom možemo uvesti sledeću formalizaciju. Nek je dat skup sekvenci $Dna$ i ceo broj $d$. Kažemo da je $k$-gram $s$  $(k,d)$- motiv  za kolekciju sekvenci $Dna$ ako se $s$ pojavljuje u svakoj sekvenci iz $Dna$ sa najviše $d$ propusta. Kanonski motiv je $(k,d)$-motiv koji nije mutirao ni na jednoj poziciji. Motivi koji se pojavljuju u niskama sa najviše $d$ propusta u odnosu na kanonski motiv nazivamo instancama kanonskog motiva.


Na primer, na Slici [2.2] za datu kolekciju nukeotidnih sekvenci, AAAAAAAAGGGGGGG je $(k,d)$-motiv. Navedeni $(k,d)$-motiv se pojavljuje kroz instance, na primer AgAAgAAAGGttGGG i  cAAtAAAAcGGcGGG, a kanonski motiv se ne pojavljuje ni u jednoj nukleotidnoj sekvenci. Dakle, kanonski motiv se ne mora pojaviti ni u jednoj sekvenci. 

[2.2]: #fig:bio2

# Prva formulacija<a id="par:prvafor"></a>

Prva formulacija problema pronalaženja motiva je predstavljena Problemom [1].  


[1]: #prob:prob1

<blockquote id="prob:prob1">

<b>Problem 1</b>: Problem nalaženja motiva, formulacija 1<br>
<i>Naći sve $(k,d)$-motive u skupu niski.</i><br>
<b>Ulaz</b>: Skup niski $Dna$, celi brojevi $k$ (dužina motiva) i $d$(maksimalan broj razlika).<br>
<b>Izlaz</b>: Svi $(k,d)$-motivi u $Dna$.

</blockquote>




Najpre ćemo pokušati da dati problem rešimo grubom silom. Pretraga grubom silom je algoritamska tehnika koja ispituje sve moguće kandidate za rešenje nekog problema. Takvi algoritmi garantuju tačno rešenje, ali je za njihovo izvršavanje potrebno dosta vremena, jer broj kandidata može biti veliki. 


# Rešenje 1

Iako pretraga grubom silom podrazumeva ispitivanje svih mogućih kandidata, očigledno je da nema potrebe ispitivati sve moguće podniske nad azbukom \{A,C,G,T\}. Svaki $(k,d)$-motiv se može razlikovati na najviše  $d$ pozicija od njegove instance u nekoj od sekvenci skupa $Dna$. Stoga možemo suziti prostor pretrage i  generisati sve takve $k$-grame i proveriti koji od njih su $(k,d)$-motivi. Ovaj algoritam se naziva algoritam $Motif$ $Enumeration$ i njegova implementacija je data ispod.

In [14]:
def MotifEnumeration(Dna,k,d):
    #rezulat smeštamo u patterns
    patterns=set()
    #prolazimo kroz sekvence iz Dna i iz svake sekvence biramo redom podsekvence dužine k
    for sequence in Dna:
        for i in range(len(sequence)-k+1):
            pattern=sequence[i:i+k]
            neighbors = set()
            #funkcija immediate_neighbors vraća susede trenutnog pattern-a
            neighbors=iterative_neighbors(pattern,d)
            # prolazimo redom kroz listu suseda
            print('Susedi pattern-a '+pattern+":")
            for neighbor in neighbors:
                print(neighbor)
                count = 0
                # i prolazimo kroz sekvence skupa Dna redom 
                for sequence in Dna:
                    for j in range(len(sequence) - k + 1):
                        # proveravamo da li se izabrani k-gram u trenutnoj sekvenci  i trenutni sused  razlikuju na <= d mesta
                        #kada nađemo jedan takav k-gram, prelazimo na sledeću sekvencu
                        seq_kmer = sequence[j:j+k]
                        # ako se razlikuju <=d mesta, onda  uvećavamo count za 1
                        if hamming_distance(seq_kmer, neighbor) <= d:
                            count += 1
                            break
                # prosli smo kroz sve sekvence, ukoliko je count jednak broju sekvenci u Dna, tog suseda dodajemo u patterns
                if count == len(Dna):
                    patterns.add(neighbor) 
                    print('DODAT U LISTU:'+neighbor)
                else:
                     print('NIJE DODAT U LISTU: '+neighbor)
    return list(patterns)


In [15]:
#funkcija koja računa Hamingovo rastojanje između dve sekvence

def hamming_distance(pattern_1, pattern_2):
    # n je dužina prve sekvence
    n = len(pattern_1)
    # rezultat čuvamo u promenjivoj distance, inicijalizujemo na 0
    distance = 0
    # prolazimo redom kroz sekvence i proveravamo da li se na istim pozicijama nalaze isti nukleotidi, 
    # ukoliko se nukelotidi razlikuju uvecavamo promenjivu distance za 1
    for i in range(n):
        if pattern_1[i] != pattern_2[i]:
            distance += 1
            
    return distance

In [16]:
# funkcija koja vraća listu suseda pattern-a
def generate_neighborhood(pattern):
    # u listu dodajemo najpre sam taj pattern
    neighborhood = set([pattern])
    # n je dužina pattern-a
    n = len(pattern)
    # prolazimo redom kroz nukleotide
    for i in range(n):
        #u promenjivu curr_nucleotide smeštamo trenutni nukleotid
        curr_nucleotide = pattern[i]
        
        for nucleotide in ['A','T','C','G']: #svaki nukleotid možemo zameniti sa preostala tri 
            if curr_nucleotide != nucleotide: # nećemo da menjamo trenutni nukleotid sa samim sobom
                new_pattern_list = list(pattern)   # pretvaramo pattern u listu karaktera       
                new_pattern_list[i] = nucleotide        # menjamo nukleotid na i-toj poziciji
                new_pattern = ''.join(new_pattern_list) # pretvaramo listu karaktera u nisku
                neighborhood.add(new_pattern) # dodajemo dobijeni pattern u rezultat 
    #vraćamo listu suseda           
    return list(neighborhood)

In [17]:
def iterative_neighbors(pattern, d):
    neighborhood = set([pattern])
    for _ in range(d):
        for neighbor in neighborhood:
            new_neigbors = generate_neighborhood(neighbor)
            neighborhood = set(list(neighborhood) + new_neigbors)
            
    return list(neighborhood)

In [18]:
input = ['TCTGAGCTTGCGTTATTTTTAGACC', 'GTTTGACGGGAACCCGACGCCTATA', 'TTTTAGATTTCCTCAGTCCACTATA','CTTACAATTTCGTTATTTATCTAAT','CAGTAGGAATAGCCACTTTGTTGTA','AAATCCATTAAGGAAAGACGACCGT']
pattern = MotifEnumeration(input,3,1)
print(pattern)

Susedi pattern-a TCT:
TCT
DODAT U LISTU:TCT
GCT
NIJE DODAT U LISTU: GCT
TGT
DODAT U LISTU:TGT
ACT
DODAT U LISTU:ACT
CCT
DODAT U LISTU:CCT
TCG
DODAT U LISTU:TCG
TCA
DODAT U LISTU:TCA
TAT
DODAT U LISTU:TAT
TTT
DODAT U LISTU:TTT
TCC
DODAT U LISTU:TCC
Susedi pattern-a AAT:
CAT
DODAT U LISTU:CAT
AAT
DODAT U LISTU:AAT
ACT
DODAT U LISTU:ACT
GAT
DODAT U LISTU:GAT
AGT
NIJE DODAT U LISTU: AGT
AAA
DODAT U LISTU:AAA
ATT
DODAT U LISTU:ATT
TAT
DODAT U LISTU:TAT
AAC
DODAT U LISTU:AAC
AAG
DODAT U LISTU:AAG
Susedi pattern-a ATC:
GTC
DODAT U LISTU:GTC
AGC
DODAT U LISTU:AGC
CTC
DODAT U LISTU:CTC
TTC
DODAT U LISTU:TTC
ATG
DODAT U LISTU:ATG
ATT
DODAT U LISTU:ATT
ATA
DODAT U LISTU:ATA
AAC
DODAT U LISTU:AAC
ACC
DODAT U LISTU:ACC
ATC
DODAT U LISTU:ATC
Susedi pattern-a TCC:
TCT
DODAT U LISTU:TCT
TGC
DODAT U LISTU:TGC
TTC
DODAT U LISTU:TTC
CCC
NIJE DODAT U LISTU: CCC
GCC
NIJE DODAT U LISTU: GCC
TAC
DODAT U LISTU:TAC
TCG
DODAT U LISTU:TCG
TCA
DODAT U LISTU:TCA
ACC
DODAT U LISTU:ACC
TCC
DODAT U LISTU:TCC
Susedi p

NIJE DODAT U LISTU: GGC
GGA
NIJE DODAT U LISTU: GGA
GGG
NIJE DODAT U LISTU: GGG
TGA
DODAT U LISTU:TGA
CGA
DODAT U LISTU:CGA
Susedi pattern-a GAA:
GTA
DODAT U LISTU:GTA
GCA
DODAT U LISTU:GCA
GAT
DODAT U LISTU:GAT
TAA
DODAT U LISTU:TAA
GAA
DODAT U LISTU:GAA
AAA
DODAT U LISTU:AAA
GAG
NIJE DODAT U LISTU: GAG
GGA
NIJE DODAT U LISTU: GGA
GAC
DODAT U LISTU:GAC
CAA
NIJE DODAT U LISTU: CAA
Susedi pattern-a AAA:
AAT
DODAT U LISTU:AAT
AGA
DODAT U LISTU:AGA
TAA
DODAT U LISTU:TAA
AAA
DODAT U LISTU:AAA
GAA
DODAT U LISTU:GAA
ACA
DODAT U LISTU:ACA
ATA
DODAT U LISTU:ATA
AAC
DODAT U LISTU:AAC
AAG
DODAT U LISTU:AAG
CAA
NIJE DODAT U LISTU: CAA
Susedi pattern-a AAG:
AAT
DODAT U LISTU:AAT
ATG
DODAT U LISTU:ATG
TAG
DODAT U LISTU:TAG
ACG
DODAT U LISTU:ACG
GAG
NIJE DODAT U LISTU: GAG
CAG
DODAT U LISTU:CAG
AAA
DODAT U LISTU:AAA
AGG
NIJE DODAT U LISTU: AGG
AAC
DODAT U LISTU:AAC
AAG
DODAT U LISTU:AAG
Susedi pattern-a AGA:
AGC
DODAT U LISTU:AGC
AGA
DODAT U LISTU:AGA
AAA
DODAT U LISTU:AAA
ACA
DODAT U LISTU:ACA
TGA


In [38]:
k=5
d=2
input = ['TCTGAGCTTGCGTTATTTTTAGACC', 'GTTTGACGGGAACCCGACGCCTATA', 'TTTTAGATTTCCTCAGTCCACTATA','CTTACAATTTCGTTATTTATCTAAT','CAGTAGGAATAGCCACTTTGTTGTA','AAATCCATTAAGGAAAGACGACCGT']
patterns = MotifEnumeration(input, k, d)
print(patterns)

Susedi pattern-a TCTGA:
TACGA
DODAT U LISTU:TACGA
TCGCA
NIJE DODAT U LISTU: TCGCA
TCAGC
NIJE DODAT U LISTU: TCAGC
TCGGC
DODAT U LISTU:TCGGC
CCTGA
NIJE DODAT U LISTU: CCTGA
TATCA
DODAT U LISTU:TATCA
ACTGG
NIJE DODAT U LISTU: ACTGG
TCTAC
DODAT U LISTU:TCTAC
TTCGA
DODAT U LISTU:TTCGA
TCGGT
DODAT U LISTU:TCGGT
TGTTA
DODAT U LISTU:TGTTA
TCGTA
NIJE DODAT U LISTU: TCGTA
TCTCT
DODAT U LISTU:TCTCT
ACTCA
DODAT U LISTU:ACTCA
TCGAA
DODAT U LISTU:TCGAA
CCTCA
DODAT U LISTU:CCTCA
GCTAA
DODAT U LISTU:GCTAA
CCTGG
NIJE DODAT U LISTU: CCTGG
TCATA
DODAT U LISTU:TCATA
TATTA
DODAT U LISTU:TATTA
TCAGG
NIJE DODAT U LISTU: TCAGG
ACCGA
NIJE DODAT U LISTU: ACCGA
TGTGA
NIJE DODAT U LISTU: TGTGA
TCTGC
NIJE DODAT U LISTU: TCTGC
TAGGA
DODAT U LISTU:TAGGA
GCTGC
NIJE DODAT U LISTU: GCTGC
TGTGG
DODAT U LISTU:TGTGG
TCTGA
NIJE DODAT U LISTU: TCTGA
TGCGA
DODAT U LISTU:TGCGA
TCAGA
DODAT U LISTU:TCAGA
TCTAG
DODAT U LISTU:TCTAG
TCCGG
NIJE DODAT U LISTU: TCCGG
TGAGA
DODAT U LISTU:TGAGA
TGTGC
NIJE DODAT U LISTU: TGTGC
TGGGA
NI

TAGGT
NIJE DODAT U LISTU: TAGGT
AAGTA
NIJE DODAT U LISTU: AAGTA
AAAGA
NIJE DODAT U LISTU: AAAGA
AAGCC
NIJE DODAT U LISTU: AAGCC
ATGAA
NIJE DODAT U LISTU: ATGAA
AGGAA
NIJE DODAT U LISTU: AGGAA
AATGC
DODAT U LISTU:AATGC
AGGGG
NIJE DODAT U LISTU: AGGGG
AAGAG
NIJE DODAT U LISTU: AAGAG
AAGGG
NIJE DODAT U LISTU: AAGGG
AAGCG
NIJE DODAT U LISTU: AAGCG
CAGAA
DODAT U LISTU:CAGAA
GATGA
NIJE DODAT U LISTU: GATGA
ACGGA
NIJE DODAT U LISTU: ACGGA
CATGA
DODAT U LISTU:CATGA
AAAGG
NIJE DODAT U LISTU: AAAGG
GAGCA
NIJE DODAT U LISTU: GAGCA
AAGTT
NIJE DODAT U LISTU: AAGTT
AAGAC
NIJE DODAT U LISTU: AAGAC
ATGGG
NIJE DODAT U LISTU: ATGGG
CGGGA
NIJE DODAT U LISTU: CGGGA
CAGGG
NIJE DODAT U LISTU: CAGGG
CAGGC
NIJE DODAT U LISTU: CAGGC
ATGGA
DODAT U LISTU:ATGGA
AATGA
NIJE DODAT U LISTU: AATGA
GAGGA
NIJE DODAT U LISTU: GAGGA
GAGGT
NIJE DODAT U LISTU: GAGGT
GAGGC
NIJE DODAT U LISTU: GAGGC
AGTGA
DODAT U LISTU:AGTGA
ACTGA
DODAT U LISTU:ACTGA
TAGAA
DODAT U LISTU:TAGAA
AAGGA
NIJE DODAT U LISTU: AAGGA
TTGGA
DODAT U LIST

NIJE DODAT U LISTU: CACGC
GAATA
NIJE DODAT U LISTU: GAATA
GGCGC
NIJE DODAT U LISTU: GGCGC
GACGC
NIJE DODAT U LISTU: GACGC
GTCGC
DODAT U LISTU:GTCGC
GACCG
NIJE DODAT U LISTU: GACCG
CACAA
NIJE DODAT U LISTU: CACAA
GACGA
NIJE DODAT U LISTU: GACGA
AGCGA
NIJE DODAT U LISTU: AGCGA
GAAGG
NIJE DODAT U LISTU: GAAGG
GACAA
NIJE DODAT U LISTU: GACAA
GACTA
NIJE DODAT U LISTU: GACTA
GTGGA
NIJE DODAT U LISTU: GTGGA
CACGT
DODAT U LISTU:CACGT
TATGA
DODAT U LISTU:TATGA
GACCA
DODAT U LISTU:GACCA
GGCTA
DODAT U LISTU:GGCTA
AACGG
NIJE DODAT U LISTU: AACGG
GTCGG
NIJE DODAT U LISTU: GTCGG
Susedi pattern-a ACGAC:
GCGGC
NIJE DODAT U LISTU: GCGGC
ACTTC
DODAT U LISTU:ACTTC
CAGAC
DODAT U LISTU:CAGAC
TTGAC
DODAT U LISTU:TTGAC
CCGAG
NIJE DODAT U LISTU: CCGAG
TCGGC
DODAT U LISTU:TCGGC
TCTAC
DODAT U LISTU:TCTAC
AGGGC
NIJE DODAT U LISTU: AGGGC
GAGAC
NIJE DODAT U LISTU: GAGAC
TCGAA
DODAT U LISTU:TCGAA
ATGAG
NIJE DODAT U LISTU: ATGAG
GCGTC
DODAT U LISTU:GCGTC
ACTAG
DODAT U LISTU:ACTAG
ACGAT
DODAT U LISTU:ACGAT
ACCAT
DODA

DODAT U LISTU:TTCCT
CTCCC
NIJE DODAT U LISTU: CTCCC
TTCCG
NIJE DODAT U LISTU: TTCCG
ATTGA
DODAT U LISTU:ATTGA
ATGTA
DODAT U LISTU:ATGTA
ATTCT
NIJE DODAT U LISTU: ATTCT
AGCGA
NIJE DODAT U LISTU: AGCGA
CTCTA
DODAT U LISTU:CTCTA
ATACC
DODAT U LISTU:ATACC
ACCAA
NIJE DODAT U LISTU: ACCAA
AGGCA
NIJE DODAT U LISTU: AGGCA
GTCCC
NIJE DODAT U LISTU: GTCCC
GACCA
DODAT U LISTU:GACCA
ATCTG
DODAT U LISTU:ATCTG
ATCAA
NIJE DODAT U LISTU: ATCAA
ACACA
DODAT U LISTU:ACACA
ATTCA
DODAT U LISTU:ATTCA
Susedi pattern-a TCCAT:
TCCAT
DODAT U LISTU:TCCAT
CCCAG
NIJE DODAT U LISTU: CCCAG
TTCAT
DODAT U LISTU:TTCAT
GCAAT
DODAT U LISTU:GCAAT
TACTT
NIJE DODAT U LISTU: TACTT
TCCAG
DODAT U LISTU:TCCAG
TACAG
DODAT U LISTU:TACAG
TGCTT
DODAT U LISTU:TGCTT
TCTAC
DODAT U LISTU:TCTAC
TTCAG
NIJE DODAT U LISTU: TTCAG
TCGGT
DODAT U LISTU:TCGGT
CCTAT
DODAT U LISTU:CCTAT
GCCTT
DODAT U LISTU:GCCTT
TCTCT
DODAT U LISTU:TCTCT
TCGAA
DODAT U LISTU:TCGAA
TCCCC
NIJE DODAT U LISTU: TCCCC
ACCTT
DODAT U LISTU:ACCTT
GTCAT
NIJE DODAT U LISTU: 

DODAT U LISTU:CCGAA
AGAAA
DODAT U LISTU:AGAAA
ATGTA
DODAT U LISTU:ATGTA
AGCAC
NIJE DODAT U LISTU: AGCAC
AGCGA
NIJE DODAT U LISTU: AGCGA
GCGAA
NIJE DODAT U LISTU: GCGAA
AGGAG
NIJE DODAT U LISTU: AGGAG
ACCAA
NIJE DODAT U LISTU: ACCAA
AGGCA
NIJE DODAT U LISTU: AGGCA
AGTAG
NIJE DODAT U LISTU: AGTAG
AGTAA
DODAT U LISTU:AGTAA
ATCAA
NIJE DODAT U LISTU: ATCAA
CGGCA
DODAT U LISTU:CGGCA
ACGTA
NIJE DODAT U LISTU: ACGTA
AGAAG
DODAT U LISTU:AGAAG
ACGAC
DODAT U LISTU:ACGAC
CGGTA
DODAT U LISTU:CGGTA
Susedi pattern-a GGAAA:
TGACA
DODAT U LISTU:TGACA
GGATC
DODAT U LISTU:GGATC
AGCAA
NIJE DODAT U LISTU: AGCAA
AAAAA
NIJE DODAT U LISTU: AAAAA
GCAAT
DODAT U LISTU:GCAAT
TTAAA
DODAT U LISTU:TTAAA
CCAAA
NIJE DODAT U LISTU: CCAAA
CGATA
DODAT U LISTU:CGATA
GGTAT
DODAT U LISTU:GGTAT
CGAAC
NIJE DODAT U LISTU: CGAAC
GTATA
DODAT U LISTU:GTATA
GGCCA
NIJE DODAT U LISTU: GGCCA
CGACA
DODAT U LISTU:CGACA
GAAAC
NIJE DODAT U LISTU: GAAAC
GGCAG
NIJE DODAT U LISTU: GGCAG
GCTAA
DODAT U LISTU:GCTAA
GAACA
DODAT U LISTU:GAACA
GG

...

NIJE DODAT U LISTU: CAGCG
CTGGA
NIJE DODAT U LISTU: CTGGA
CTGCA
DODAT U LISTU:CTGCA
CCGCA
NIJE DODAT U LISTU: CCGCA
CGGTC
DODAT U LISTU:CGGTC
CTGTG
NIJE DODAT U LISTU: CTGTG
CCGGA
NIJE DODAT U LISTU: CCGGA
CGCTA
DODAT U LISTU:CGCTA
AAGTA
NIJE DODAT U LISTU: AAGTA
GGGTA
NIJE DODAT U LISTU: GGGTA
CATTT
DODAT U LISTU:CATTT
CACTC
NIJE DODAT U LISTU: CACTC
TAGTC
DODAT U LISTU:TAGTC
CATTC
NIJE DODAT U LISTU: CATTC
CGGAA
NIJE DODAT U LISTU: CGGAA
CACCA
NIJE DODAT U LISTU: CACCA
TACTA
DODAT U LISTU:TACTA
CAGAA
DODAT U LISTU:CAGAA
CAGCC
NIJE DODAT U LISTU: CAGCC
CATGA
DODAT U LISTU:CATGA
CGGTT
DODAT U LISTU:CGGTT
GAGCA
NIJE DODAT U LISTU: GAGCA
AAGTT
NIJE DODAT U LISTU: AAGTT
CAGAG
NIJE DODAT U LISTU: CAGAG
CGGGA
NIJE DODAT U LISTU: CGGGA
CAGGG
NIJE DODAT U LISTU: CAGGG
CAGGC
NIJE DODAT U LISTU: CAGGC
CATTA
DODAT U LISTU:CATTA
GAGGA
NIJE DODAT U LISTU: GAGGA
CAAAA
NIJE DODAT U LISTU: CAAAA
TAGAA
DODAT U LISTU:TAGAA
AAGGA
NIJE DODAT U LISTU: AAGGA
GAGAA
DODAT U LISTU:GAGAA
CACTG
NIJE DODAT U LIS

DODAT U LISTU:ATAAA
GCAGA
NIJE DODAT U LISTU: GCAGA
GTAAA
DODAT U LISTU:GTAAA
CAAAA
NIJE DODAT U LISTU: CAAAA
GGAGA
NIJE DODAT U LISTU: GGAGA
GTAGA
DODAT U LISTU:GTAGA
GGTGA
NIJE DODAT U LISTU: GGTGA
GAGAA
DODAT U LISTU:GAGAA
GTGAA
DODAT U LISTU:GTGAA
GGATT
DODAT U LISTU:GGATT
TGTAA
DODAT U LISTU:TGTAA
GGACG
NIJE DODAT U LISTU: GGACG
GAAGA
NIJE DODAT U LISTU: GAAGA
GTAAC
NIJE DODAT U LISTU: GTAAC
GGGAA
NIJE DODAT U LISTU: GGGAA
CTAAA
NIJE DODAT U LISTU: CTAAA
CGCAA
DODAT U LISTU:CGCAA
GGAAG
NIJE DODAT U LISTU: GGAAG
GGTAA
NIJE DODAT U LISTU: GGTAA
GAAAT
DODAT U LISTU:GAAAT
GGGAG
NIJE DODAT U LISTU: GGGAG
GGTTA
DODAT U LISTU:GGTTA
AGAGA
NIJE DODAT U LISTU: AGAGA
GGAAC
NIJE DODAT U LISTU: GGAAC
GAAAG
NIJE DODAT U LISTU: GAAAG
GGCAA
NIJE DODAT U LISTU: GGCAA
AGAAT
DODAT U LISTU:AGAAT
GTAAT
NIJE DODAT U LISTU: GTAAT
GCAAG
NIJE DODAT U LISTU: GCAAG
GCACA
NIJE DODAT U LISTU: GCACA
GGCAT
NIJE DODAT U LISTU: GGCAT
GGATA
DODAT U LISTU:GGATA
GGGGA
NIJE DODAT U LISTU: GGGGA
TCAAA
NIJE DODAT U LIS

AAGGC
NIJE DODAT U LISTU: AAGGC
GTATC
DODAT U LISTU:GTATC
AGATC
DODAT U LISTU:AGATC
Susedi pattern-a AATCC:
AATCC
DODAT U LISTU:AATCC
AAGGC
NIJE DODAT U LISTU: AAGGC
ACTTC
DODAT U LISTU:ACTTC
AGTCG
DODAT U LISTU:AGTCG
AGTGC
NIJE DODAT U LISTU: AGTGC
CATAC
DODAT U LISTU:CATAC
CACCC
NIJE DODAT U LISTU: CACCC
AATGG
NIJE DODAT U LISTU: AATGG
AACTC
DODAT U LISTU:AACTC
AGTCT
DODAT U LISTU:AGTCT
CTTCC
NIJE DODAT U LISTU: CTTCC
TATCA
DODAT U LISTU:TATCA
AATAA
NIJE DODAT U LISTU: AATAA
AGCCC
NIJE DODAT U LISTU: AGCCC
AATCA
NIJE DODAT U LISTU: AATCA
TTTCC
DODAT U LISTU:TTTCC
GATCA
DODAT U LISTU:GATCA
TAGCC
DODAT U LISTU:TAGCC
ACTCA
DODAT U LISTU:ACTCA
AGACC
NIJE DODAT U LISTU: AGACC
GATGC
NIJE DODAT U LISTU: GATGC
AATTG
DODAT U LISTU:AATTG
AATGT
NIJE DODAT U LISTU: AATGT
GATTC
DODAT U LISTU:GATTC
GATCC
DODAT U LISTU:GATCC
AATAT
DODAT U LISTU:AATAT
TACCC
DODAT U LISTU:TACCC
CCTCC
DODAT U LISTU:CCTCC
GAGCC
NIJE DODAT U LISTU: GAGCC
TAACC
DODAT U LISTU:TAACC
TATCC
DODAT U LISTU:TATCC
ACTCT
DODAT U 

DODAT U LISTU:AAGAA
CTGGA
NIJE DODAT U LISTU: CTGGA
ACGGC
NIJE DODAT U LISTU: ACGGC
TGGGA
NIJE DODAT U LISTU: TGGGA
AGGGT
NIJE DODAT U LISTU: AGGGT
CCGGA
NIJE DODAT U LISTU: CCGGA
TAGGT
NIJE DODAT U LISTU: TAGGT
AAGTA
NIJE DODAT U LISTU: AAGTA
AAAGA
NIJE DODAT U LISTU: AAAGA
AAGCC
NIJE DODAT U LISTU: AAGCC
ATGAA
NIJE DODAT U LISTU: ATGAA
AGGAA
NIJE DODAT U LISTU: AGGAA
AATGC
DODAT U LISTU:AATGC
AGGGG
NIJE DODAT U LISTU: AGGGG
AAGAG
NIJE DODAT U LISTU: AAGAG
AAGGG
NIJE DODAT U LISTU: AAGGG
AAGCG
NIJE DODAT U LISTU: AAGCG
CAGAA
DODAT U LISTU:CAGAA
GATGA
NIJE DODAT U LISTU: GATGA
ACGGA
NIJE DODAT U LISTU: ACGGA
CATGA
DODAT U LISTU:CATGA
AAAGG
NIJE DODAT U LISTU: AAAGG
GAGCA
NIJE DODAT U LISTU: GAGCA
AAGTT
NIJE DODAT U LISTU: AAGTT
AAGAC
NIJE DODAT U LISTU: AAGAC
ATGGG
NIJE DODAT U LISTU: ATGGG
CGGGA
NIJE DODAT U LISTU: CGGGA
CAGGG
NIJE DODAT U LISTU: CAGGG
CAGGC
NIJE DODAT U LISTU: CAGGC
ATGGA
DODAT U LISTU:ATGGA
AATGA
NIJE DODAT U LISTU: AATGA
GAGGA
NIJE DODAT U LISTU: GAGGA
GAGGT
NIJE D

AGAAC
DODAT U LISTU:AGAAC
ACGCC
NIJE DODAT U LISTU: ACGCC
AAAAC
NIJE DODAT U LISTU: AAAAC
ACCTC
DODAT U LISTU:ACCTC
ATGTC
DODAT U LISTU:ATGTC
AAGAA
DODAT U LISTU:AAGAA
ACGGC
NIJE DODAT U LISTU: ACGGC
CGGAC
DODAT U LISTU:CGGAC
ATTAC
DODAT U LISTU:ATTAC
TGGAC
NIJE DODAT U LISTU: TGGAC
ACGTG
NIJE DODAT U LISTU: ACGTG
ACCGC
NIJE DODAT U LISTU: ACCGC
ACAAT
NIJE DODAT U LISTU: ACAAT
GCGAC
NIJE DODAT U LISTU: GCGAC
ACGTC
DODAT U LISTU:ACGTC
GCGAG
NIJE DODAT U LISTU: GCGAG
GGGAC
NIJE DODAT U LISTU: GGGAC
AGGTC
DODAT U LISTU:AGGTC
ACGTT
DODAT U LISTU:ACGTT
CTGAC
NIJE DODAT U LISTU: CTGAC
ATGAC
DODAT U LISTU:ATGAC
TAGAC
DODAT U LISTU:TAGAC
AAGCC
NIJE DODAT U LISTU: AAGCC
ACGCT
DODAT U LISTU:ACGCT
AGTAC
DODAT U LISTU:AGTAC
ATGAA
NIJE DODAT U LISTU: ATGAA
GCAAC
NIJE DODAT U LISTU: GCAAC
AGGAA
NIJE DODAT U LISTU: AGGAA
AGGCC
NIJE DODAT U LISTU: AGGCC
CCGCC
NIJE DODAT U LISTU: CCGCC
ACAAC
DODAT U LISTU:ACAAC
AAGAG
NIJE DODAT U LISTU: AAGAG
TCGCC
DODAT U LISTU:TCGCC
ACGGA
NIJE DODAT U LISTU: ACGGA
AC

In [39]:
#primer sa ROSALIND
k=3
d=1
input=['ATTTGGC','TGCCTTA','CGGTATC','GAAAATT']
patterns = MotifEnumeration(input, k, d)
print(patterns)

Susedi pattern-a ATT:
CTT
NIJE DODAT U LISTU: CTT
GTT
DODAT U LISTU:GTT
TTT
DODAT U LISTU:TTT
ACT
NIJE DODAT U LISTU: ACT
AAT
NIJE DODAT U LISTU: AAT
ATA
DODAT U LISTU:ATA
AGT
NIJE DODAT U LISTU: AGT
ATT
DODAT U LISTU:ATT
ATG
NIJE DODAT U LISTU: ATG
ATC
NIJE DODAT U LISTU: ATC
Susedi pattern-a AAA:
AGA
NIJE DODAT U LISTU: AGA
GAA
NIJE DODAT U LISTU: GAA
AAG
NIJE DODAT U LISTU: AAG
ACA
NIJE DODAT U LISTU: ACA
AAA
NIJE DODAT U LISTU: AAA
CAA
NIJE DODAT U LISTU: CAA
AAC
NIJE DODAT U LISTU: AAC
ATA
DODAT U LISTU:ATA
AAT
NIJE DODAT U LISTU: AAT
TAA
NIJE DODAT U LISTU: TAA
Susedi pattern-a AAA:
AGA
NIJE DODAT U LISTU: AGA
GAA
NIJE DODAT U LISTU: GAA
AAG
NIJE DODAT U LISTU: AAG
ACA
NIJE DODAT U LISTU: ACA
AAA
NIJE DODAT U LISTU: AAA
CAA
NIJE DODAT U LISTU: CAA
AAC
NIJE DODAT U LISTU: AAC
ATA
DODAT U LISTU:ATA
AAT
NIJE DODAT U LISTU: AAT
TAA
NIJE DODAT U LISTU: TAA
Susedi pattern-a AAT:
AAG
NIJE DODAT U LISTU: AAG
CAT
NIJE DODAT U LISTU: CAT
ACT
NIJE DODAT U LISTU: ACT
AAA
NIJE DODAT U LISTU: 

Ukoliko imamo $t$ sekvenci dužine $n$, vremenska složenost algoritma je O($n$ $\cdot$ $4^k$ $\cdot$
$t$).Mana ovog algoritma je da je prilično spor za velike $k$ i $d$, iako smo redukovali broj kandidata i njegovih instanci. Pored toga, neće uvek važiti da svaka sekvenca iz $Dna$ sadrži instancu $(k,d)$-motiva, jer se u skupu sekvenci mogu naći geni koji nisu pod kontrolom odgovarajućih transkripcionih faktora. 

# Rešenje 2

Ovaj pristup podrazumeva da unutar kolekcije $Dna$ odredimo dva najsličnija $k$-grama koji pripadaju različitim sekvencama. Postupak je sledeći:
- poredimo parove niski iz kolekcije $Dna$
- uočimo dva najsličnija $k$-grama u dve niske iz $Dna$
- napravimo kanonski motiv 
- proverimo da li je to $(k,d)$-motiv.


Dobijene niske se od kanonskog motiva mogu razlikovati na $d$ pozicija, a među sobom na $2$ $\cdot$ $d$. Na primer, dva $k$-grama AgAAgAAAGGttGGG i cAAtAAAAcGGGGcG se razlikuju od AAAAAAAAGGGGGGG na 4 pozicije, ali međusobno imaju čak 8  od 15 nepodudaranja (Slika [2.3]).  

[2.3]: #fig:bio3

<figure><img src="bioinformatika3.png" width="27%" id="fig:bio3" /><figcaption style="text-align: center;"><b>Slika 2.3</b>: Primer dva $15$-grama čija je distanca od kanonskog $15$-grama jednaka $4$, a međusobna distanca jednaka $8$ </figcaption></figure>


Na $benchmark$ kolekciji je ovim pristupom pronađeno nekoliko hiljada parova $k$-grama koji su se razlikovali na manje od $8$ pozicija. Zbog toga ovaj pristup nije dobar izbor za rešavanje problema nalaženja motiva.

# Druga formulacija <a id="par:drugafor"></a>

U prethodnoj formulaciji ideja je bila da određujemo jednu po jednu instancu motiva u niskama kolekcije $Dna$. Nova formulacija podrazumeva određivanje cele kolekcije motiva odjednom. Za ovakav pristup je neophodno imati funkciju koja procenjuje koliko su nađeni motivi međusobno slični. 

Posmatramo $t$ sekvenci skupa $Dna$ svaki dužine $n$ i slučajnim izborom izaberemo $k$-gram iz svake sekvence kako bismo formirali kolekciju motiva $Motifs$, koju predstavljamo $t \times k$ matricom motiva. Primer takve matrice se može videti na Slici [2.4]. U svakoj koloni velikim slovima ćemo predstavljati najzastupljeniji nukleotid. Ukoliko u koloni postoji više najzastupljenijih nukeotida, onda proizvoljno izaberemo jedan. Variranjem izbora $k$-grama u svakoj sekvenci, možemo konstruisati veliki broj različitih matrica motiva od sekvenci skupa $Dna$. Cilj je naći takve $k$-grame koje će rezultirati matricom sa najviše velikih slova. 

[2.4]: #fig:bio4


Konsenzus niska za kolekciju $Motifs$ je sastavljena od najzastupljenijih nukleotida u svakoj koloni  i označava se sa $Consensus(Motifs)$ (Slika [2.4]).

[2.4]: #fig:bio4

Da bismo među više kolekcija motiva izabrali najbolju, neophodno je definisati funkciju pomoću koje bi se mogao oceniti kvalitet kolekcije motiva. Jedan od načina da se ta funkcija zada je kao zbir najmanje zastupljenih  nukelotida u svakoj koloni u oznaci $Score(Motifs)$. Na Slici [2.4] vidimo da druga i poslednja kolona u kolekciji motiva $Motifs$ doprinose skoru $Score(Motifs)$ po  4. 


[2.4]: #fig:bio4

<figure><img src="bioinformatika11dod.png" width="50%" id="fig:bio4" /><figcaption style="text-align: center;"><b>Slika 2.4</b>: Primer kolekcije motiva, konsenzus niske i skora matrice  </figcaption></figure>



Nova ideja podrazumeva nalaženje kolekcije $Motifs$ koja minimizuje $Score(Motifs)$, tj. kolekcije u kojoj je broj manje zastupljenih nukleotida  što manji i zahteva drugačiju definiciju problema pronalaženja motiva (Problem [2]).

[2]: #prob:prob2

<blockquote id="prob:prob2">

<b>Problem 2</b>: Problem nalaženja motiva, formulacija 2<br>
<i>Za dati skup sekvenci, pronaći skup $k$-grama, po jedan iz svake sekvence sa minimalnim skorom među svim mogućim takvim skupovima.</i><br>
<b>Ulaz</b>: Skup sekvenci $Dna$, ceo broj $k$.<br>
<b>Izlaz</b>: Skup $k$ − grama $Motifs$, po jedan iz svake sekvence skupa
$Dna$, tako da je vrednost $Score(Motifs)$ minimalna.

</blockquote>

Pokušajmo da rešimo ovaj problem primenom grube sile. Algoritam grube sile za prolanaženja motiva  razmatra svaki mogući izbor $k$-grama $Motifs$ iz $Dna$ (jedan $k$-gram iz svake sekvence od $n$ nukelotida) i vraća kolekciju $Motifs$ sa minimalnim rezultatom. Možemo izabrati $k$-gram u svakoj od $t$ sekvenci na $n-k+1$ načina, pa postoji $(n-k+1)^t$ načina da se formira skup motiva $Motifs$. Za svaki izbor $Motifs$, računa se $Score(Motfis)$ u $k  \cdot t$ koraka. Ako pretpostavimo da je $k$ mnogo manje od $n$, vremenska složenost algoritma je O($n^t$  $\cdot$ k  $\cdot$ t). U $benchmark$ kolekciji $n$ = $600$, $t$=$10$ i $k$ =$15$, ali u realnim primerima ovi parametri mogu biti i veći. Algoritam je dosta spor, pa je potrebno da reformulišemo problem pronalaženja motiva.  .  

# Treća formulacija

U prethodnom rešenju smo ispitivali sve moguće kolekcije motiva i za svaku određivali konsenzus nisku. U novom rešenju ideja je obrnuta, tj. podrazumeva da istražimo sve potencijalne $k$-grame za konsenzus nisku i onda u datoj kolekciji sekvenci da pronađemo kolekciju motiva koja joj najviše odgovara. Da bismo izračunali koliko jedna niska odgovara kolekciji motiva, potrebna nam je funkcija koja to ocenjuje.

Na Slici [2.5] vidimo da se predloženi $Score(Motifs)$ može računati i red po red. Svaki sabirak u ovom zbiru predstavlja propuste ($mismatches$) između $Consensus(Motifs)$ i $k$-grama u odgovarajućem redu matrice motiva, tj. Hamingovo rastojanje između ovih niski. Svaka vrednost na kraju reda odgovara Hamingovom rastojanju između tog reda i konsenzus niske.  Hamingovo rastojanje između dva $k$-grama je broj pozicija na kojima se razlikuju.

[2.5]:#fig:bio5

<figure><img src="bioinformatika10.png" width="60%" id="fig:bio5" /><figcaption style="text-align: center;"><b>Slika 2.5</b>:Računanje skora matrice red-po-red</figcaption></figure>

Ukoliko imamo niz $k-grama$ $Motifs=\{Motif_1,...,Motif_t\}$ i $k-gram$ $Pattern$ definišemo
$d(Pattern, Motifs)$ kao sumu Hamingovih rastojanaja između Pattern i svakog $Motif_i$,
$$
    d(Pattern,Motifs)=\sum_{i=1}^{t} HammingDistance(Pattern,Motif_i)
$$
Možemo zaključiti da skor matrice  motiva ($Score(Motifs)$) odgovara rastojanju konsenzus niske od matrice motiva   
($d(Consensus(Motifs),Motifs)$).


U novom pristupu, umesto da tražimo kolekciju $k$-grama $Motifs$ koja minimizuje $Score(Motifs)$, tražićemo potencijalnu konsenzus nisku $Paterrn$ koja minimizuje $d(Pattern,Motifs)$ među svim mogućim izborima $Pattern$ i svim mogućim izborima skupa $k$-grama $Motifs$ iz $Dna$. Problemom [3] je predstavljena nova formulacija problema pronalaženja motiva.

[3]: #prob:prob3

<blockquote id="prob:prob3">

<b>Problem 3</b>: Problem nalaženja motiva, formulacija 3<br>
<i>Na osnovu skupa sekvenci $Dna$, naći $Pattern$ i kolekciju $k$-grama $Motifs$ po jedan iz svake sekvence takav da je $d(Pattern,Motifs)$ minimalan.</i><br>
<b>Ulaz</b>: Skup sekvenci $Dna$, ceo broj $k$.<br>
<b>Izlaz</b>: $k$-gram $Pattern$ i skup $k$-grama $Motifs$ koji minimizuje $d(Pattern,Motifs)$ među svim mogućim izborima $Pattern$ i $Motifs$.

</blockquote>

Analizirajmo formulaciju 3 u odnosu na formulaciju 2. U formulaciji 2 minimizujemo $Score(Motifs)$, a u formulaciji 3 $d(Pattern, Motifs)$. Umesto minimizacije funkcije po jednoj promenljivoj($Score(Motifs)$), minimizujemo funkciju po dve promenljive ($d(Pattern, Motifs)$). Ima smisla postaviti pitanje da li je formulacija 3 zapravo teži problem od formulacije 2. U formulaciji 3, promenljive $Pattern$ i $Motifs$ nisu nezavisne i stoga nema potrebe da uzimamo u obzir sve moguće niske $Pattern$ i sve moguće kolekcije $Motifs$ kako bismo minimizovali $d(Pattern, Motifs)$. Umesto toga, za fiksiranu nisku $Pattern$ možemo birati elemente kolekcije $Motifs$ jedan po jedan, iz svake niske, koji će minimizovati $d(Pattern, Motifs)$ za fiksiranu nisku $Pattern$. Da bismo ilustrovali ovu ideju, definišemo $Motifs(Pattern,Dna)$ kao kolekciju $k$-grama koja minimizuje $d(Pattern, Motifs)$ za dati $Pattern$ i sve moguće kolekcije $Motifs$ iz $Dna$. Na Slici [2.6] je  prikazano $5$ obojenih $3$-grama koji predstavljaju $Motifs(AAA,Dna)$.

[2.6]: #fig:bio6

<figure><img src="motivi.png" width="20%" id="fig:bio6" /><figcaption style="text-align: center;"><b>Slika 2.6</b>: Kolekcija motiva $Dna$ sa označenim 3-gramima koji predstavljaju $Motifs(AAA, Dna)$</figcaption></figure>

Umesto da za fiksirano $Pattern$ ispitujemo sve moguće kolekcije $Motifs$, možemo generisati $k$-grame po jedan iz svake sekvence koji će minimizovati rastojanje, tj. možemo izabrati $k$-gram iz svakog reda nezavisno od ostalih redova. Za izbor ovakvog $k$-grama potrebno je da definišemo Hamingovo rastojanje dve niske različitih dužina.

Hamingovo rastojanje između dve niske različitih dužina u oznaci $d(Pattern,Text)$, predstavlja minimum Hamingovih rastojanja između $k$-grama $Pattern$ i svih $k$-grama u  dužoj niski $Text$, tj.
$$
    d(Pattern,Text)= \min_{\forall  k-gram  Pattern^{'}  iz  Text} HammingDistance(Pattern, Pattern’)
$$

Na Slici [2.7] je dat postupak računanja $d(GATTCTCA, GCAAAGACGCTGACCAA)$. Zaključujemo da je $d(GATTCTCA, GCAAAGACGCTGACCAA)$ = $3$.

[2.7]: #fig:bio7

<figure><img src="primer1.jpg" width="70%" id="fig:bio7" /><figcaption style="text-align: center;"><b>Slika 2.7</b>:Postupak računanja $d(GATTCTCA, GCAAAGACGCTGACCAA)$ </figcaption></figure>

Definišemo rastojanje između $k$-grama $Pattern$ i skupa niski $Dna=\{Dna_{1},...,Dna_{t}\}$ u oznaci $d(Pattern,Dna)$ kao  zbir rastojanja između $Pattern$ i svake niske skupa $Dna$, tj.
$$
    d(Pattern,Dna)=\sum_{i=1}^{t} d(Pattern,Dna_i)
$$

Za sekvence  $Dna$ prikazane na Slici [2.8]  važi $d(AAA,Dna)=1+1+2+0+1=5$.


[2.8]: #fig:bio8

<figure><img src="bioinformatika12.png" width="20%" id="fig:bio8" /><figcaption style="text-align: center;"><b>Slika 2.8</b>: Kolekcija sekvenci $Dna$ sa vrednostima $d(AAA,Dna)$ za i=1,..,5</figcaption></figure>

# Problem niske medijane <a id="par:niskamedijana"></a>

Kao što smo videli, da bismo minimizovali funkciju $d(Pattern, Motifs)$, dovoljno je da nađemo $k$-gram $Pattern$ koji minimizuje $d(Pattern,Dna)$ među svim $k$-gramima $Pattern$. Takav $k$-gram zovemo niska medijana (Problem [4]). 

[4]: #prob:prob4

<blockquote id="prob:prob4">

<b>Problem 4</b>: Problem niske medijane<br>
<i>Naći nisku medijanu.</i><br>
<b>Ulaz</b>: Skup sekvenci $Dna$, ceo broj $k$.<br>
<b>Izlaz</b>:$k$-gram  $k$-mer  koji minimuzuje rastojanje $d(k$-mer,$Dna)$.

</blockquote>

Algoritam $Median$ $String$ za rešavanje problema niske medijane je dat ispod. Ovaj algoritam izračunava $d(k$-mer, $Dna)$ za svaki od $4^k$ $k$-grama i  kao rezultat će vratiti $k$-gram koji ima najmanje rastojanje od skupa $Dna$. Problem koji se javlja pri implementaciji algoritma $Median$ $String$ jeste pisanje funkcije $d(Pattern,Dna)$ kao zbir rastojanja između $Pattern$ i svake sekvence u $Dna$.

In [19]:
def MedianString(Dna,k): 
    #promenjivu dist inicijalizujemo na beskonačno.
    dist = float('inf')
    #rezultat smeštamo u promenjivu median_pattern
    median_pattern = ''
    #Generišemo sve moguće k-grame
    for i in range(4 ** k): #svaki k-gram može biti jedan od 4^k mogućih kombinacija nukleotida
        pattern = number_to_pattern(i, k)
        #proveravamo ukupno rastojanje pattern-a i skupa Dna
        curr_dist = d(pattern,Dna)
        print('Potencijalni median string je:'+pattern+'. Ukupno rastojanje tog k-grama od Dna je:'+str(curr_dist))
        #Ažuriramo trenutni median_pattern ako je pronađen k-gram sa manjim ukupnim rastojanjem

        if curr_dist < dist:
            dist = curr_dist
            median_pattern = pattern
    return median_pattern

In [20]:
#funkcija koja pretvara broj u k-gram
def number_to_pattern(number,k):
    # ako je k==1, vraćamo rezultat funkcije number_to_symbol
    if k == 1:
        return number_to_symbol(number)
    # ako je k != 1, u promenjivu r smeštamo ostatak broja pri deljenju sa 4
    # a u promenjivu q broj koji dobijamo pri deljenju broja sa 4
    r = number % 4
    q = number // 4
    # u promenjivu last_sym smeštamo rezultat poziva funkcije  number_to_symbol(r)
    last_sym = number_to_symbol(r)
    
    # poziva se rekurzivno number_to_pattern, ali kao prvi argument prosleđujemo q, a za drugi k umanjeno za 1
    # i dodamo last_sym
    return number_to_pattern(q, k - 1) + last_sym

In [21]:
#funkcija koja pretvara broj u nukleotid: O u A, 1 u T, 2 u C i 3 u G
def number_to_symbol(number):
    mapping = {
        0: 'A',
        1: 'T',
        2: 'C',
        3: 'G'
    }
    #Ukoliko prosleđena vrednost nije 0,1,2 ili 3 ispisujemo grešku
    if number not in [0,1,2,3]:
        print(f'Pogrešan unos!')
        return None
    
    return mapping[number]

In [22]:
#funkcija računa ukupno rastojanje između k-grama pattern i skupa sekvenci Dna

def d(pattern, dna):
    # k je dužina pattern-a
    k = len(pattern)
    # rezultat smeštamo u promenjivu total_dist, inicijalizujemo na 0
    total_dist = 0
    
    #prolazimo redom kroz sekvence
    for sequence in dna:
        # n je dužina sekvence
        n = len(sequence)
        # promenjivu min_dist inicijalizujemo na beskonačno
        min_dist = float('inf')
        
        #za svaku sekvencu iz Dna računa se Hamingovo rastojanje između k-grama pattern  i svih mogućih k-grama u toj sekvenci
        # za svaku sekvencu uzima se minimalno rastojanje i dodaje se na ukupno rastojanje
        for i in range(0, n - k + 1):
            curr_pattern = sequence[i : i + k]
            curr_dist = hamming_distance(pattern, curr_pattern.upper())
            #ako je izračunato rastojanje manje od min_dist, ažuriramo min_dist
            if curr_dist < min_dist:
                min_dist = curr_dist
        # dodajemo na total_dist min_dist, i prelazimo na sledeću sekvencu       
        total_dist += min_dist
    return total_dist

In [23]:
dna = [
    'ttaccttAAC',
    'gATAtctgtc',
    'ACGgcgttcg',
    'ccctAAAgag',
    'cgtcAGAggt'
]

pattern ='AAA'
d(pattern, dna)

5

In [24]:
dna = [
    'ttaccttAAC',
    'gATAtctgtc',
    'ACGgcgttcg',
    'ccctAAAgag',
    'cgtcAGAggt'
]
k = 3
MedianString(dna, k)

Potencijalni median string je:AAA. Ukupno rastojanje tog k-grama od Dna je:5
Potencijalni median string je:AAT. Ukupno rastojanje tog k-grama od Dna je:7
Potencijalni median string je:AAC. Ukupno rastojanje tog k-grama od Dna je:6
Potencijalni median string je:AAG. Ukupno rastojanje tog k-grama od Dna je:5
Potencijalni median string je:ATA. Ukupno rastojanje tog k-grama od Dna je:5
Potencijalni median string je:ATT. Ukupno rastojanje tog k-grama od Dna je:7
Potencijalni median string je:ATC. Ukupno rastojanje tog k-grama od Dna je:5
Potencijalni median string je:ATG. Ukupno rastojanje tog k-grama od Dna je:6
Potencijalni median string je:ACA. Ukupno rastojanje tog k-grama od Dna je:5
Potencijalni median string je:ACT. Ukupno rastojanje tog k-grama od Dna je:6
Potencijalni median string je:ACC. Ukupno rastojanje tog k-grama od Dna je:5
Potencijalni median string je:ACG. Ukupno rastojanje tog k-grama od Dna je:5
Potencijalni median string je:AGA. Ukupno rastojanje tog k-grama od Dna je:5

'CCT'

In [25]:
#Primer sa ROSALIND
k=3
dna=['AAATTGACGCAT',
'GACGACCACGTT',
'CGTCAGCGCCTG',
'GCTGAGCACCGG',
'AGTACGGGACAG']
MedianString(dna, k)

Potencijalni median string je:AAA. Ukupno rastojanje tog k-grama od Dna je:7
Potencijalni median string je:AAT. Ukupno rastojanje tog k-grama od Dna je:7
Potencijalni median string je:AAC. Ukupno rastojanje tog k-grama od Dna je:5
Potencijalni median string je:AAG. Ukupno rastojanje tog k-grama od Dna je:5
Potencijalni median string je:ATA. Ukupno rastojanje tog k-grama od Dna je:8
Potencijalni median string je:ATT. Ukupno rastojanje tog k-grama od Dna je:6
Potencijalni median string je:ATC. Ukupno rastojanje tog k-grama od Dna je:6
Potencijalni median string je:ATG. Ukupno rastojanje tog k-grama od Dna je:5
Potencijalni median string je:ACA. Ukupno rastojanje tog k-grama od Dna je:4
Potencijalni median string je:ACT. Ukupno rastojanje tog k-grama od Dna je:5
Potencijalni median string je:ACC. Ukupno rastojanje tog k-grama od Dna je:3
Potencijalni median string je:ACG. Ukupno rastojanje tog k-grama od Dna je:2
Potencijalni median string je:AGA. Ukupno rastojanje tog k-grama od Dna je:5

'ACG'

Za izačunavanje rastojanja između $k$-grama i jedne niske skupa $Dna$ potrebno je $k$ $\cdot$ $(n-k+1)$ koraka. Kako imamo $t$ niski, za izračunavanje rastojanja $k$-grama od skupa potrebno nam je  $t$ $\cdot$ $k$ $\cdot$ $(n-k+1)$, tj. približno $t$ $\cdot$ $k$ $\cdot$ $n$ koraka. Zaključujemo da algoritam $Median$ $String$ ima vremensku složenost O($4^k$ $\cdot$ k $\cdot$ n $ \cdot$ t). 

Možemo uporediti vremensku složenost algoritma grube sile i algoritma $Median$ $String$. Algoritam $Median$ $String$ se u praksi pokazao bolje od algoritma grube sile (vremenska složenost O($n^t \cdot k \cdot t$)), jer dužina motiva $k$ ne prelazi $20$, a $t$ se meri u hiljadama. U Tabeli [2.1] prikazano je da  je algoritam MedianString testiran na $benchmark$ skupu prespor za $k$=$15$. Kada je testiran na istom skupu za $k$=$13$, kao rešenje je vratio kolekciju motiva $Motifs$ sa konsenzus niskom AAAAAtAGaGGGG, pri čemu je $Score(Motifs)$ koji je izračunat kao suma Hamingovih rastojanja između svakog motiva iz $Motifs$ i konsenzus niske jednak 29.

[2.1]: #tab:tab1

<figure>
<figcaption style="text-align: center;"><b>Тabela 2.1</b>: Rezultati testiranja algoritama na $benchmark$ skupu</figcaption>
<table id="tab:tab1">
<thead>
<tr class="header">
<th style="text-align: center;">Algoritam</th>
<th style="text-align: center;">k</th>
<th style="text-align: center;">Rešenje</th>
<th style="text-align: center;">Skor</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td style="text-align: center;">$Median$ $String$</td>
<td style="text-align: center;">13</td>
<td style="text-align: center;">AAAAAtAGaGGGG</td>
<td style="text-align: center;">29</td>

</tr>
<tr class="even">
<td style="text-align: center;">$Median$ $String$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">Previše spor</td>
<td style="text-align: center;"></td>
</tr>
<tr>
<td style="text-align: center;" colspan="4">Traženi $(k,d)$ motiv:   AAAAAAAAGGGGGGG</td>  
    
</tr>
</tbody>
</table>
</figure>

# Probabilistički algoritmi <a id="par:prob"></a>

Deterministički algoritmi daju uvek isto rešenje za isti ulaz, dok kod probabilističkih to ne mora biti slučaj zato što oni imaju bar jedan korak koji predstavlja neki slučajan izbor ($randomness$). Stoga zbog postizanja najbolje tačnosti, ove algoritme treba pokrenuti više puta i uzeti najbolji rezulat.

Da bismo analizirali neke od probabilističkih rešenja problema nalaženja motiva, neophodno nam je da definišemo matrice $Count(Motifs)$ i $Profile(Motifs)$ za skup motiva $Motifs$.
$Count(Motifs)$ je matrica dimenzije $4 \times k$ koja predstavlja broj pojavljivanja svakog nukleotida u svakoj koloni matrice motiva. Dakle, element matrice $Count(Motifs)$  na poziciji $(i,j)$ predstavlja koliko se puta nukleotid $i$ pojavljuje u koloni $j$ matrice $Motifs$. Slika [3.1] predstavlja $Count(Motifs)$ za kolekciju motiva $Motifs$ sa Slike [2.4]. Ovo rezultira profilnom matricom u oznaci $Profile(Motifs)$, gde element ove matrice na poziciji $(i,j)$ predstavlja frekvenciju $i$-og nukleotida u koloni $j$ matrice $Motifs$. Zbir vrednosti bilo koje kolone ove matrice mora biti $1$. 

[3.1]: #fig:bio9
[2.4]: #fig:bio4

<figure><img src="bioinformatika6.png" width="40%" id="fig:bio9" /><figcaption style="text-align: center;"><b>Slika 3.1</b>:Primer $Count(Motifs)$</figcaption></figure>

Svaka kolona $Profile(Motifs)$ odgovara distribuciji verovatnoće tj. koloni nenegativnih brojeva čiji je zbir $1$. Na Slici [3.2] je dat primer profilne matrice gde vidimo da druga kolona odgovara vrednostima verovatnoća $0.2$, $0.6$, $0.0$ i $0.2$, za A, C, G  i T.

[3.2]: #fig:bio10

<figure><img src="bioinformatika7.png" width="40%" id="fig:bio10" /><figcaption style="text-align: center;"><b>Slika 3.2</b>:Primer $Profile(Motifs)$</figcaption></figure>


Neka je data kolekcija $k$-grama $Motifs$ izabranih iz $t$ sekvenci $Dna$. Profilnu matricu sa $k$ kolona možemo posmatrati kao kolekciju od $k$ kockica, koju ćemo baciti nasumično da bismo generisali $k$-gram. Na primer, ako je prva kolona profilne matrice (0.2, 0.1, 0.0, 0.7) onda generišemo A kao prvi nukleotid sa verovatnoćom $0.2$, C sa verovatnoćom $0.1$, G sa verovatnoćom $0.0$ i T sa verovatnoćom  $0.7$. 

$Pr(a|P)$ definišemo kao uslovnu verovatnoću generisanja $k$-grama $a$ za datu profilnu matricu $P$. Ako je $k$-gram $a$ sličan konsenzus niski onda je $Pr(a|P)$  visok. Generisanje pojedinačnih nukleotida je međusobno nezavisno. Ukoliko sa  $P_{a_i,i}$ označimo verovatnoću slova $a_i$ na poziciji $i$, onda se verovatnoća  $Pr(a|P)$  dobija jednostavnim množenjem  pojedinačnih verovatnoća:

$$
    Pr(a|P) = \prod_{i=1}^kP_{a_i,i}.  
$$


Neka je Tabelom [3.1] predstavljena profilna matrica Profile. 

[3.1]: #tab:tab2

<figure>
<figcaption style="text-align: center;"><b>Тabela 3.1</b>: Profile</figcaption>
<table id="tab:tab2">

<tbody>
<tr class="odd">
<td style="text-align: center;">A</td>
<td style="text-align: center;">1/2</td>
<td style="text-align: center;">7/8</td>
<td style="text-align: center;">3/8</td>
<td style="text-align: center;">0</td>
<td style="text-align: center;">1/8</td>
<td style="text-align: center;">0</td>
</tr>
<tr class="odd">
<td style="text-align: center;">C</td>
<td style="text-align: center;">1/8</td>
<td style="text-align: center;">0</td>
<td style="text-align: center;">1/2</td>
<td style="text-align: center;">5/8</td>
<td style="text-align: center;">3/8</td>
<td style="text-align: center;">0</td>
</tr>
<tr class="odd">
<td style="text-align: center;">T</td>
<td style="text-align: center;">1/8</td>
<td style="text-align: center;">1/8</td>
<td style="text-align: center;">0</td>
<td style="text-align: center;">0</td>
<td style="text-align: center;">1/4</td>
<td style="text-align: center;">7/8</td>
</tr>
<tr class="odd">
<td style="text-align: center;">G</td>
<td style="text-align: center;">1/4</td>
<td style="text-align: center;">0</td>
<td style="text-align: center;">1/8</td>
<td style="text-align: center;">3/8</td>
<td style="text-align: center;">1/4</td>
<td style="text-align: center;">1/8</td>
</tr>

</tbody>
</table>
</figure>

Koristeći profilnu matricu dobijamo da je $Pr(ATACAG|Profile)$ $=$ $\frac{1}{2}$ $\cdot$ $\frac{1}{8}$ $\cdot$ $\frac{3}{8}$ $\cdot$ $\frac{5}{8}$ $\cdot$ $\frac{1}{8}$ $\cdot$ $\frac{1}{8}$ $=$ $0.001602$.

Kada je data profilna matrica, možemo proceniti verovatnoću svakog $k$-grama u sekvenci i naći najverovatniji $k$-gram u sekvenci. Ispod je Problemom [5] predstavljen problem nalaženja najverovatnijeg $k$-grama u niski $Text$.

[5]: #prob:prob5

<blockquote id="prob:prob5">

<b>Problem 5</b>: Problem najverovatnijeg $k$-grama<br>
<i>Naći najverovatniji $k$-gram u nizu.</i><br>
<b>Ulaz</b>: Dat je niz $Text$, ceo broj $k$  i profilna matrica $4 \times k$.<br>
<b>Izlaz</b>: Najverovatniji $k$-gram u $Text$.

</blockquote>

Na osnovu profilne matrice koja je predstavljena Tabelom [3.1], računamo najverovatniji $6$-gram u niski $ctataaaccttacat$ (Slika [3.3]). Postupak je sledeći:
- uzimamo podniske dužine $6$
- za svaku podnisku a i datu profilnu matricu $P$ računamo $Pr(a|P)$
- na osnovu izračunatih verovatnoća pronalazimo najverovatniji $6$-gram.


U navedenom primeru, najverovatniji $6$-gram je $aaacct$.

[3.3]: #fig:bio11
[3.1]: #tab:tab2

<figure><img src="najverovatnijikgram.png" width="60%" id="fig:bio11" /><figcaption style="text-align: center;"><b>Slika 3.3</b>:Izračunavanje najverovatnijeg 6-grama</figcaption></figure>

Ispod je data implementacija algoritma za nalaženje najverovatnijeg $k$-grama na osnovu profilne matrice.

In [26]:
#funkcija koja računa verovatnoću k-grama pattern u odnosu na profilnu matricu
def probability(motif_profile, pattern, k):
    #rezultat smeštamo u promenjivu prob, inicijalizujemo na 1, jer je to neutral za množenje
    prob = 1
    #prolazimo redom kroz nukeotide sekvence
    for i in range(k):
        # u promenjivu symbol smeštamo trenutni nukleotid
        symbol = pattern[i]
        #pretvaramo nukleotid u broj
        j = symbol_to_number(symbol)
        #potražimo u profilnoj matrici vrednost u i-om redu i j-oj koloni
        symbol_prob = motif_profile[i][j]
        # i tu vrednost pomnožimo sa prob
        prob *= symbol_prob
    return prob

In [27]:
#funkcija koja pretvara nukeotid u broj: A u 0, C u 1, G u 2, T u 3
def symbol_to_number(symbol):
    mapping = {
        'A': 0,
        'C': 1,
        'G': 2,
        'T': 3
    }
    
    return mapping[symbol.upper()]

In [28]:
# funkcija koja računa najverovatniji k-gram u niski text
def most_probable_kmer(text,k,motif_profile):
    # promenjivu max_probability postavimo na -1
    max_probability = -1
    # u promenjivu most_probable_kmer smeštamo rezultat
    most_probable_kmer = ''
    # za svaki k-gram u niski text računamo verovatnoću tog k-grama na osnovu profilne matrice
    for i in range(len(text)-k+1):
        kmer = text[i:i+k]
        prob = probability(motif_profile,kmer,k)
        # ako je verovatnoća tog k-grama veća od max_probability, ažuriramo max_probability, i taj k-gram se postavlja na most_probable_kmer
        if prob > max_probability:
            max_probability = prob
            most_probable_kmer = kmer
    return most_probable_kmer

In [29]:
#Primer sa ROSALIND
text='ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT'
k=5
motif_profile=[
    [0.2,0.4,0.3,0.1],
    [0.2,0.3,0.3,0.2],
    [0.3,0.1,0.5,0.1],
    [0.2,0.5,0.2,0.1],
    [0.3,0.1,0.4,0.2]
]
most_probable_kmer(text,k,motif_profile)

'CCGAG'

# Pohlepna pretraga motiva  <a id="par:greedymotifsearch"></a>

Pohlepni algoritmi predstavljaju pristup rešavanju optimizacionih problema gde se u svakom koraku bira najbolja opcija koja je dostupna u tom trenutku. Pohlepni algoritmi tako biraju lokalno optimalno rešenje u svakom koraku u nadi da ce ih ti izbori dovesti do globalnog optimuma. Ova strategija ne mora uvek dovoditi do optimalnog rešenja. Na primer, ukoliko prilikom igranja šaha gledamo samo jedan potez unapred, moguće je da ce to dovesti do lošeg rezultata. Ipak, ovi algoritmi su jednostavni i često imaju malu vremensku složenost, što ih čini efikasnim za određene vrste problema. Međutim, nekad se može desiti da ne pronađu globalno optimalno rešenje.  
 

U nastavku ćemo analizirati jedan od probabilističkih algoritama za rešavanje problema nalaženja motiva, $Greedy$ $Motif$ $Search$,  koji je implementiran u nastavku. Ideja algoritma je da  postavlja svaki od $k$-grama iz $Dna_1$ (prva sekvenca iz skupa sekvenci $Dna=\{Dna_1,...,Dna_t\}$) kao  prvi motiv $Motif_1$ potencijalno traženog skupa motiva $Motifs$. Za dati izbor $k$-grama iz $Dna_1$ pravi profilnu matricu i postavlja za $Motif_2$ najverovatniji $k$-gram iz $Dna_2$. Zatim se ažurira profilna matrica na osnovu $Motif_1$ i $Motif_2$ i postavlja se za $Motif_3$ najverovatniji $k$-gram iz $Dna_3$. Nakon pronalaženja $i-1$ $k$-grama $Motifs$ u prvim $i-1$ sekvencama iz $Dna$, ovaj algoritam konstruiše profilnu matricu i bira najverovatniji $k$-gram iz $Dna_i$ na osnovu profilne matrice. Kada dobijemo skup $k$-grama $Motifs$ proverava se da li taj skup nadmašuje trenutnu kolekciju motiva sa najboljim rezultatom, a zatim se bira naredni $Motif_1$ i ponavlja se proces generisanja  $Motifs$.

In [30]:
# funkcija vraća profilnu matricu motiva
def get_profile(motifs, k,pseudo=0):
    #u promenjivu n smeštamo broj motiva
    n=len(motifs)
    # funkcija get_count vraća count matricu 
    counts = get_count(motifs, k,pseudo)
    #profile_matrix predstavlja profilnu matricu k x 4. Na početku inicijalizujemo na 0 matricu.
    profile_matrix = [[0] * 4 for _ in range(k)]
    #popunjavamo profilnu matricu redom
    for i in range(k):
        for j in range(4):
            profile_matrix[i][j] = counts[i][j]/n
    return profile_matrix

In [31]:
#funkcija vraća count matricu
def get_count(motifs,k,pseudo=0):
    #rezultat je matrica count. Matrica je k x 4, pri čemu je inicijalizovana na 0 matricu
    count=[[pseudo]*4 for _ in range(k)]
    #iteriramo kroz sve motive
    for motif in motifs:
        # i iteriramo kroz nukeotide svakog motiva iz motifs
        for i in range(k):
            #u promenivu curr_nuc smeštamo trenutni nukleotid
            curr_nuc = motif[i]
            #nukleotid pretvorimo u broj i smestimo u promenivu j
            j = symbol_to_number(curr_nuc)
            # uvećavamo matiricu na poziciji [i][j] za 1
            count[i][j]+=1
    return count

In [32]:
#funkcija računa skor skupa motiva
def get_score(motifs,k,pseudo=0):
    #rezultat smeštamo u promenjivu score, inicjalizujemo na 0
    score = 0
    #računamo count matricu
    counts =get_count(motifs, k,pseudo)
     
    #prolazeći kroz count matricu imamo uvid o manje zastupljenim nukleotidima
    for i in range(k):
        most_frequent = counts[i].index(max(counts[i]))
        for j in range(4):
            if j != most_frequent:
                 score += counts[i][j]
    
    return score

In [33]:
#funkcija računa na osnovu profilne matrice najverovatniji k-gram u sekvenci
def most_probable_kmer(motif_profile, sequence, k):
    max_prob = -1
    #rezultat smeštamo u most_probable
    most_probable = ''
    #prolazimo redom kroz svaki k-gram u sekvenci
    for i in range(len(sequence) - k + 1):
        #u promenjivu kmer smeštamo trenutni k-gram
        kmer = sequence[i:i + k]
        #računamo verovatnoću k-grama na osnovu profilne matrice
        prob = probability(motif_profile, kmer, k)
        #ako je dobijena verovatnoća veća od max_prob, ažuriramo max_prob, i taj kmer postavljamo na most_probable
        if prob > max_prob:
            max_prob = prob
            most_probable = kmer
    
    return most_probable

In [34]:
#algoritam pohlepne pretrage
def greedy_motif_search(dna, k, t):
    # u nizu best_motifs smeštamo rezulat. Na početku taj niz čine prvi k-grami iz svake sekvence dna
    best_motifs = [seq[:k] for seq in dna]
    
        
    #uzimamo prvu sekvencu
    first_seq = dna[0]
    # n je dužina prve sekvence iz dna
    n=len(first_seq)
    #iteriramo kroz sve moguće k-grame iz prve sekvence
    for i in range(n - k + 1):
        # u skup trenutnih motiva stavimo i-ti k-gram iz prve sekvence
        motifs = [first_seq[i : i + k]]
        
        for j in range(1, t): #iteriramo kroz preostale sekvence
            # pravimo profilnu matricu na osnovu trenutnih motiva
            motif_profile = get_profile(motifs, k,0)
            # na osnovu dobijene profilne matrice, računamo najverovatniji k-gram u j-oj sekvenci dna
            motif_i = most_probable_kmer(motif_profile, dna[j], k)
            
            # najverovatniji k-gram dodajemo u skup trenutnih motiva
            motifs.append(motif_i)
        
       # ako trenutni motivi imaju skor koji je manji od skora najboljih motiva
        if  get_score(motifs,k) < get_score(best_motifs,k):
            # ažuriramo best_motifs
            best_motifs = motifs
            
    return best_motifs

In [35]:
#primer sa ROSALIND
k=3
t=5
dna=[
    'GGCGTTCAGGCA',
    'AAGAATCAGTCA',
    'CAAGGAGTTCGC',
    'CACGTCAATCAC',
    'CAATAATATTCG'
]
greedy_motif_search(dna, k, len(dna))

['CAG', 'CAG', 'CAA', 'CAA', 'CAA']

In [36]:
# Primer
dna = [
    'ttACCTtaac',
    'gATGTctgtc',
    'acgGCGTtag',
    'ccctaACGAg',
    'cgtcagAGGT'
]

k = 4
greedy_motif_search(dna, k, len(dna))

['tACC', 'gATG', 'acgG', 'aACG', 'gAGG']

In [37]:
#funkcija koja generiše random sekvencu dužine 600
import random
import string
def get_random_sequence():
    length = 600
    result_str=[]
    letters = ['A','T','C','G']
    result_str = ''.join(random.choice(letters) for i in range(600))
    return result_str


#funkcija koja generiše 10 nukleotidnih sekvenci dužine 600
import random
def get_random_ten_sequence():
    random.seed(0)
    sequences = []
    for i in range(10):
         sequences.append(get_random_sequence())
    return sequences
        

In [38]:
#funkcija koja u svaku nukleotidnu sekvencu na random mesto ubacuje motiv AAAAAAAAGGGGGGG
from random import randrange
def get_insert_motif_in_sequence():

    new_sequence = None
    sequences =  get_random_ten_sequence()
    new_sequences=[]
    for sequence in sequences:
        #print('*********')
        motif = mutation_motif('AAAAAAAAGGGGGGG')
        n = len(motif)
        #print(motif)
        random_index = randrange(len(sequence)-n+1)
        new_sequence = sequence[:random_index] + motif + sequence [random_index+n:]
        #print(new_sequence)
        
        new_sequences.append(new_sequence)
    return new_sequences
       



In [39]:
from random import randrange
def mutation_motif(sequence):
    sequence_new= list(sequence)
    indeksi_za_promenu = random.sample(range(len(sequence_new)), 4)
    for index in indeksi_za_promenu:
        sequence_new[index] = get_nucleotide()
        
    sequence_rez = ''.join(sequence_new)
    return sequence_rez

In [40]:
import random
def get_nucleotide():
    slova = 'ACGT'
    novo_slovo = random.choice(slova)
    return novo_slovo


In [41]:
dna = get_insert_motif_in_sequence()
greedy_motif_search(dna, 15, len(dna))

['GGAAGCCGAGTATAG',
 'GTGCAACAACGGGAC',
 'TAGCTGACAAACATG',
 'TGTGCTCCTTGCTCA',
 'GGGATGACTTGCTTG',
 'GGGGGGAGTGGCATG',
 'TAAGGGACAAGCTTG',
 'GGGATGCGTAGATTG',
 'GGACGTAGTTGCTTC',
 'GGGGGGCGTGGGGTA']

<figure><img src="bioinformatika14.png" width="16%" id="fig:bio12" /><figcaption style="text-align: center;"><b>Slika 3.4</b>:Kolekcija sekvenci $Dna$</figcaption></figure>

Želimo da pomoću ovog algoritma nađemo $(4,1)$-motiv $ACGT$ u kolekciji sekvenci $Dna$ sa Slike [3.4], pri čemu su velikim slovima u svakoj sekvenci predstavljene instance $(4,1)$-motiva $ACGT$, plavim slovima su predstavljeni nukleotidi koji se poklapaju sa motivom, a crvenim oni koji se ne poklapaju sa motivom  $ACGT$.
Pretpostavimo da je algoritam izabrao iz prve sekvence $ACCT$ i konstruišemo profilnu matricu (Slika [3.5]).

[3.4]: #fig:bio12
[3.5]: #fig:bio13

<figure><img src="bioinformatika15.png" width="20%" id="fig:bio13" /><figcaption style="text-align: center;"><b>Slika 3.5</b>:Profilna matrica</figcaption></figure>

Sada algoritam treba da nađe najverovatniji $k$-gram u drugoj sekvenci. Problem predstavlja to što u profilnoj matrici postoje nule, tako da je verovatnoća svakog $4$-grama $0$, osim $ACCT$. Dakle, ako $ACCT$ nije prisutan u svakoj sekvenci, mala je šansa da će ovaj algoritam vratiti rešenje. U nastavku ćemo pokazati kako se Laplasovo pravilo sukcesije može iskoristiti u prevazilaženju ovog problema.

Razmotrimo performanse algoritma $Greedy$ $Motif$. U odnosu na algoritam $Median$ $String$, ovaj algoritam je brži. Za razliku od algoritma $Median$ $String$, algoritam $Greedy Motif$ uspeva da pronađe rešenje za $k=15$. U Tabeli [3.2] je prikazan rezultat algoritma $Greedy$ $Motif$  testiran na $benchmark$ skupu. Kao rezultat vraća kolekciju motiva $Motifs$ sa konsenzus niskom gtAAAtAgaGatGtG, pri čemu je $Score(Motifs)$ = $29$, što znači da daje lošije rešenje od algoritma $Median$ $String$.

[3.2]: #tab:tab3

<figure>
<figcaption style="text-align: center;"><b>Тabela 3.2</b>: Rezultati testiranja algoritama na $benchmark$ skupu</figcaption>
<table id="tab:tab3">
<thead>
<tr class="header">
<th style="text-align: center;">Algoritam</th>
<th style="text-align: center;">k</th>
<th style="text-align: center;">Rešenje</th>
<th style="text-align: center;">Skor</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td style="text-align: center;">$Median$ $String$</td>
<td style="text-align: center;">13</td>
<td style="text-align: center;">AAAAAtAGaGGGG</td>
<td style="text-align: center;">29</td>

</tr>
<tr class="even">
<td style="text-align: center;">$Median$ $String$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">Previše spor</td>
<td style="text-align: center;"></td>
</tr>
<tr class="even">
<td style="text-align: center;">$Greedy$ $Motif$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">gtAAAtAgaGatGtG</td>
<td style="text-align: center;">58</td>
</tr>
<tr>
<td style="text-align: center;" colspan="4">Traženi $(k,d)$ motiv: AAAAAAAAGGGGGGG</td>   
</tr>
</tbody>
</table>
</figure>

# Poboljšana pohlepna pretraga motiva  <a id="par:poboljsanapohlepnapret"></a>

Na Slici [3.6] je prikazana uslovna verovatnoća generisanja $TCGTGGATTTCC$ za datu profilnu matricu $Profile$. Možemo videti da četvrti simbol u $TCGTGGATTTCC$  dovodi da $Pr(TCGTGGATTTCC | Profile )$ bude jednak $0$. Niska $TCGTGGATTTCC$  se razlikuje od konsenzus niske $TCGGGGATTTCC$ na samo jednoj poziciji. $TCGTGGATTCC$ ima istu malu verovatnoću kao $AAATCTTGGAA$, koja se veoma razlikuje od konsenzus niske. 


[3.6]: #fig:bio14

<figure><img src="bioinformatika16.png" width="60%" id="fig:bio14" /><figcaption style="text-align: center;"><b>Slika 3.6</b>:Pr(TCGTGGATTTCC | Profile) </figcaption></figure>

Jedan od načina da se ovaj fenomen bolje modeluje jeste da se nule zamene malim vrednostima koje se nazivaju pseudovrednosti. Postupak uvođenja pseudovrednosti naziva se Laplasovo pravilo sukcesije. Naime, ukoliko ponovimo neki eksperiment $n$ puta i dobijemo $s$ puta uspeh, potrebno je izračunati verovatnoću da sledeći eksperiment ima uspešan ishod. 
Uvodimo oznake $X_{1}$, ..., $X_{n+1}$ (uslovno nezavisne logičke promenjive) pri čemu je $X_{i}$ jednak 1 ako je $i$-ti eksperiment imao uspešan ishod ili 0 ako je neuspešan. Tada važi:
$$
Pr(X_{n+1}=1|X_{1}+…+X_{n}=s )= \cfrac{s}{n}.
$$
Kako znamo da su u sledećem eksperimentu dva moguća ishoda (tačno/netačno), onda smo obavili $n+2$ eksperimenta sa $s+1$ uspešnim ishodom. Dodate vrednost predstavljaju pseudovrednosti i važi:
$$
Pr(X_{n+1}=1|X_{1}+…+X_{n}=s )= \cfrac{s+1}{n+2}.
$$



Da bismo primenili Laplasovo pravilo sukcesije na problem nalaženja motiva, posmatrajmo kolekciju motiva $Motifs$ i nad njom generisane matrice $Count(Motifs)$ i $Profile(Motifs)$ (Slika [3.7]).

[3.7]: #fig:bio15

<figure><img src="bioinformatika17.png" width="60%" id="fig:bio15" /><figcaption style="text-align: center;"><b>Slika 3.7</b>:Matrice bez primene Laplasovog pravila</figcaption></figure>


Kako Laplasovo pravilo sukcesije dodaje svakom elementu matrice $Count(Motifs)$ $1$, na Slici [3.8] se mogu videti ažurirane $Count(Motifs)$ i $Profile(Motifs)$.


[3.8]: #fig:bio16

<figure><img src="bioinformatika18.png" width="60%" id="fig:bio16" /><figcaption style="text-align: center;"><b>Slika 3.8</b>:Matrice  nakon primene Laplasovog pravila</figcaption></figure>

Jedina promena koju treba da uvedemo algoritmu $Greedy$ $Motif$ $Search$ da bismo eliminisali nule je da svakom polju matrice $Count(Motifs)$ dodamo neku vrednost. Ispod je data implementacija $Greedy$ $Motif$ $Search$ koristeći pseudovrednosti. 


In [53]:
#jedina promena u odnosu na algoritam GreedyMotifSearch je da pri  računanju Count(Motifs) 
#svakom polju se doda neka vrednost, 
#kako bi se izbeglo da neko polje ima vrednost 0

def GreedyMotifSearchwithPseudocounts(dna,k,t,pseudo):
 

    # u nizu best_motifs smeštamo rezultat. Na početku taj niz čine prvi k-grami iz svake sekvence dna
    best_motifs = [seq[:k] for seq in dna]
    
        
    #uzimamo prvu sekvencu
    first_seq = dna[0]
    # n je dužina prve sekvence iz dna
    n=len(first_seq)
    #iteriramo kroz sve moguće k-grame iz prve sekvence
    for i in range(n - k + 1):
        # u skup trenutnih motiva stavimo i-ti k-gram iz prve sekvence
        motifs = [first_seq[i : i + k]]
        
        for j in range(1, t): #iteriramo kroz preostale sekvence
            # pravimo profilnu matricu na osnovu trenutnih motiva
            motif_profile = get_profile(motifs, k,pseudo)
            # na osnovu dobijene profilne matrice, računamo najverovatniji k-gram u j-oj sekvenci dna
            motif_i = most_probable_kmer(motif_profile, dna[j], k)
            
            # najverovatniji k-gram dodajemo u skup trenutnih motiva
            motifs.append(motif_i)
        
       # ako trenutni motivi imaju skor koji je manji od skora najboljih motiva
        if  get_score(motifs,k,pseudo) < get_score(best_motifs,k,pseudo):
            # ažuriramo best_motifs
            best_motifs = motifs
            
    return best_motifs

In [54]:
k=3
t=5
#prosleđujemo pseudocount kao četvrti argument funkcije GreedyMotifSearchwithPseudocounts
pseudo=4
dna=[
    'GGCGTTCAGGCA',
    'AAGAATCAGTCA',
    'CAAGGAGTTCGC',
    'CACGTCAATCAC',
    'CAATAATATTCG'
]
GreedyMotifSearchwithPseudocounts(dna,k,t,pseudo)

['TTC', 'ATC', 'TTC', 'ATC', 'TTC']

In [69]:
dna = get_insert_motif_in_sequence()
GreedyMotifSearchwithPseudocounts(dna, 15, len(dna),1)

['AATATAAGAAAGGGG',
 'GCTACAAGGGAGGGG',
 'GATAAAAAGGGGGGG',
 'AAGAAAAAGTCGGGG',
 'ATCAAAAAATAGGGG',
 'AATTAAAGGGGGGGG',
 'TATAAAAGAAGTGGG',
 'AATATAAAAGGGGGA',
 'AACAAAAAGGGGGGT',
 'AATAAACGGGGGGGG']

Ilustrovaćemo primenu poboljšanog pohlepnog algoritma na primeru pronalaženja motiva  unutar kolekcije sekvenci $Dna$ (Slika [3.4]).

Skup motiva gradimo dodavajući jedan po jedan motiv i pretpostavimo da je algoritam izabrao iz prve niske $ACCT$. Sada konstruišemo $Count(Motifs)$ i $Profile(Motifs)$ koristeći Laplasovo pravilo sukcesije (Slika [3.9]).


[3.4]: #fig:bio12
[3.9]: #fig:bio17

<figure><img src="bioinformatika20.png" width="60%" id="fig:bio17" /><figcaption style="text-align: center;"><b>Slika 3.9</b>:Prvi korak algoritma</figcaption></figure>


Koristimo profilnu matricu da izračunamo verovatnoće svih $4$-grama iz druge niske.

<figure><img src="bioinformatika21.png" width="60%" id="fig:bio18" /><figcaption style="text-align: center;"><b>Slika 3.10</b>:Verovatnoće svih 4-grama iz druge niske </figcaption></figure>

Sa Slike [3.10] vidimo da su dva najverovatnija $4$-grama iz druge niske ATGT i GTct. Izaberimo $ATGT$.
Na Slici  [3.11] su data dva motiva $Motifs$, $Count(Motifs)$ i $Profile(Motifs)$.

[3.10]: #fig:bio18
[3.11]: #fig:bio19

<figure><img src="bioinformatika22.png" width="60%" id="fig:bio19" /><figcaption style="text-align: center;"><b>Slika 3.11</b>:Drugi korak algoritma  </figcaption></figure>

Koristimo dobijenu profilnu matricu da izračunamo verovatnoće svih $4$-grama iz treće niske (Slika [3.12]).

[3.12]: #fig:bio20

<figure><img src="bioinformatika23a.png" width="60%" id="fig:bio20" /><figcaption style="text-align: center;"><b>Slika 3.12</b>:Verovatnoće svih $4$-grama iz treće niske   </figcaption></figure>

Vidimo na Slici [3.12] da su dva najverovatnija $4$-grama iz treće niske acgG i GCGT. Biramo $acgG$. Skupu motiva $Motifs$ dodamo $acgG$ i ažuriramo $Count(Motifs)$ i $Profile(Motifs)$ (Slika [3.13]).

[3.12]: #fig:bio20
[3.13]: #fig:bio21

<figure><img src="bioinformatika23b.png" width="60%" id="fig:bio21" /><figcaption style="text-align: center;"><b>Slika 3.13</b>:Treći korak algoritma </figcaption></figure>


Koristimo profilnu matricu da izračunamo verovatnoće svih $4$-grama iz četvrte niske (Slika [3.14]).

[3.14]: #fig:bio22

<figure><img src="bioinformatika25.png" width="50%" id="fig:bio22" /><figcaption style="text-align: center;"><b>Slika 3.14</b>:Verovatnoće svih $4$-grama iz četvrte niske </figcaption></figure>


Najverovatniji $4$-gram u četvrtoj niski je $ACGA$. Dodajemo u $Motifs$ motiv $ACGA$ i ažuriramo $Count(Motifs)$ i $Profile(Motifs)$ (Slika [3.15]).


[3.15]: #fig:bio23

<figure><img src="bioinformatika26.png" width="60%" id="fig:bio23" /><figcaption style="text-align: center;"><b>Slika 3.15</b>:Četvrti korak algoritma  </figcaption></figure>

Najzad, računamo verovatnoće svih $4$-grama iz pete niske (Slika [3.16]).

[3.16]: #fig:bio24

<figure><img src="bioinformatika27.png" width="60%" id="fig:bio24" /><figcaption style="text-align: center;"><b>Slika 3.16</b>:Verovatnoće svih $4$-grama pete niske Dna   </figcaption></figure>

Najverovatniji $4$-gram u petoj niski je \texttt{AGGT}. Algoritam  $Greedy$ $Motif$ $Search$ je proizveo matricu motiva $Motifs$, koja podrazumeva ispravan $Consensus(Motifs)$ ACGT (Slika [3.17]).

[3.17]: #fig:bio25


<figure><img src="bioinformatika28.png" width="30%" id="fig:bio25" /><figcaption style="text-align: center;"><b>Slika 3.17</b>:Skup motiva $Motifs$ koji je vratio algoritam i $Consensus(Motifs) </figcaption></figure>


Algoritam $Median$ $String$ je prespor za $k>13$, a algoritam $Greedy$ $Motif$ je brži i može da nađe motive dužine $k=15$. Veća brzina donosi manju tačnost.
U Tabeli [3.3] je prikazano da je algoritam $Greedy$ $Motif$ primenom Laplasovog pravila  testiran na $benchmark$ kolekciji vratio kolekciju motiva $Motifs$ sa konsenzus niskom AAAAAtAgaGGGGtt, pri čemu je $Score(Motifs)$ = $41$.


[3.3]: #tab:tab4

<figure>
<figcaption style="text-align: center;"><b>Тabela 3.3</b>: Rezultati testiranja algoritama na $benchmark$ skupu</figcaption>
<table id="tab:tab4">
<thead>
<tr class="header">
<th style="text-align: center;">Algoritam</th>
<th style="text-align: center;">k</th>
<th style="text-align: center;">Rešenje</th>
<th style="text-align: center;">Skor</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td style="text-align: center;">$Median$ $String$</td>
<td style="text-align: center;">13</td>
<td style="text-align: center;">AAAAAtAGaGGGG</td>
<td style="text-align: center;">29</td>

</tr>
<tr class="even">
<td style="text-align: center;">$Median$ $String$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">Previše spor</td>
<td style="text-align: center;"></td>
</tr>
<tr class="even">
<td style="text-align: center;">$Greedy$ $Motif$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">gtAAAtAgaGatGtG</td>
<td style="text-align: center;">58</td>
</tr>
<tr class="even">
<td style="text-align: center;">$Greedy$ $Motif$ $Laplace$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">AAAAAtAgaGGGGtt</td>
<td style="text-align: center;">41</td>
</tr>
<tr>
<td style="text-align: center;" colspan="4">Traženi $(k,d)$ motiv: AAAAAAAAGGGGGGG</td>   
</tr>
</tbody>
</table>
</figure>

# Probabilistička pretraga motiva  <a id="par:randomizovanapretragamotiva"></a>

Prethodno opisani algoritam  $Greedy$ $Motif$ $Search$ uvek kreće od istog skupa motiva. Ideja je da umesto kretanja od početnih pozicija, krećemo od nekih nasumično izabranih. 
Kako ne bismo iterirali kroz niske kao u prethodnom algoritmu, možemo pokušati da pravimo profilnu matricu od skupa motiva i obrnuto i da te korake ponavljamo sve dok se skor motiva smanjuje. U nastavku će biti prikazan algoritam $Randomized$ $Motif$ $Search$ čija je ideja da se ažuriranjem profilne matrice i matrice motiva dobija što verovatniji skup motiva. 


Pre nego što analiziramo pomenuti algoritam, potrebno je dati sledeću formalizaciju. Neka je dat skup sekvenci $Dna$ i profilna matrica $Profile$ dimenzije $4 \times k $. Definišemo $Motifs(Profile,Dna)$ kao kolekciju $k$-grama formiranu od najverovatnijih $k$-grama iz svake sekvence skupa $Dna$.

Možemo početi sa nasumično izabranim $k$-gramima motiva iz  kolekcije sekvenci $Dna$,  konstruisati matricu $Profile(Motifs)$, a zatim iskoristiti tu matricu da bi se konsturisala nova kolekcija $k$-grama $Motifs$:
$$
    Motifs(Profile (Motifs),Dna).
$$
Dobijamo $Motifs$, a zatim konstruišemo profilnu matricu:
$$
    Profile(Motif(Profile(Motifs), Dna)) 
$$
koju koristimo da bismo dobili najverovatnije $k$-grame:
$$
    Motifs(Profile(Motifs(Profile(Motifs), Dna)) , Dna).
$$
Nastavljamo dalju iteraciju:
$$
    . . . Profile ( Motifs ( Profile ( Motifs (Profile (Motifs), Dna) ) , Dna )) . . .
$$
sve dok se skor poboljšava.

Ovaj algoritam počinjemo sa nasumično izabranim $k$-gramima pa je potreban generator slučajnih brojeva ($Random(N)$) koji jednako verovatno može vratiti bilo koji ceo broj između $1$ i $N$. Ispod je data implementacija algoritma $Randomized$ $Motif$ $Search$.

In [70]:
import random

def RandomizedMotifSearch(dna, k, t):
    motifs= []
    #nasumično izaberemo k-grame iz sveke sekvence dna i smestimo u motifs
    for seq in dna:
        i = random.randrange(0, len(seq) - k + 1)
        motifs.append(seq[i : i + k])
    #rezultat smeštamo u best_motifs. Na početku u best_motifs smeštamo nasumično izabrane k-grame, tj. motifs
    best_motifs = motifs
    best_score = get_score(best_motifs, k)
    
    while True:
        #računamo profilnu matricu na osnovu skupa motiva motifs
        motif_profile = get_profile(motifs, k)
        # na osnovu dobijene profilne matrice i dna, vraćamo skup motiva
        motifs = get_motifs(motif_profile, dna, k)
        #ukoliko dobijeni skup motiva ima manji skor od skora motiva best_motifs, ažuriramo best_motfis
        if get_score(motifs, k)< get_score(best_motifs, k):
            best_motifs = motifs
            best_score = get_score(motifs, k)
        else:
        #u suprotnom vratimo best_motifs
            return best_motifs, best_score

In [71]:
#funkcija na osnovu profilne matrice i skupa sekvenci dna, vraća skup motiva
def get_motifs(motif_profile, dna, k):
    #rezultat smeštamo u motifs
    motifs = []
    #iteriramo redom kroz sekvence
    for seq in dna:
        # za svaku sekvencu nađemo najverovatniji k-gram na osnovu profilne matrice
        motif = most_probable_kmer(motif_profile, seq, k)
        #dobijeni motiv dodamo u motifs
        motifs.append(motif)

    return motifs

In [72]:
#primer sa ROSALIND
k=8
t=5
dna=[
    'CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA',
    'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
    'TAGTACCGAGACCGAAAGAAGTATACAGGCGT',
    'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
    'AATCCACCAGCTCCACGTGCAATGTTGGCCTA' ]

min_score = float('inf')
min_motifs = None
#algortam pokrećemo 1000 puta i uzimamo najbolji rezultat
for _ in range(1000):
    best_motifs, best_score = RandomizedMotifSearch(dna, k, t)
    if best_score < min_score:
        min_score = best_score
        min_motifs = best_motifs
print(best_motifs)


['GTAAACGG', 'GTAAGTGC', 'GAAAGAAG', 'GATCAAGT', 'TGCAATGT']


In [74]:
dna = get_insert_motif_in_sequence()
RandomizedMotifSearch(dna, 15, len(dna))

(['CTGCGAAAATTACAG',
  'CGAAGACCTGTCTGC',
  'CTGAAAACGCAGCCG',
  'CTGAAAACTTTACAC',
  'CGCCAAGCATTGTAG',
  'CTGCAAGAATCGTCG',
  'CTTAATCCCTACCAG',
  'GGTATAGCACAGTCG',
  'ATGCTACAGGTGTGC',
  'TTAATACCGGTGACG'],
 63)

Ilustrovaćemo primenu $Randomized$ $Motif$ $Search$ algoritma na primeru pronalaženja motiva unutar kolekcije sekvenci $Dna$. Na Slici [3.18] su crvenom bojom označeni nasumično izabrani $k$-grami iz svake sekvence, a velikim slovima su predstavljene instance $(4,1)$-motiva $ACGT$.

[3.18]: #fig:bio26


<figure><img src="bioinformatika31.png" width="28%" id="fig:bio26" /><figcaption style="text-align: center;"><b>Slika 3.18</b>:Nasumično izabrani $k$-grami iz svake sekvence skupa $Dna$ </figcaption></figure>


Na osnovu $k$-grama $Motifs$ pravimo $Profile(Motifs)$ (Slika [3.19]).

[3.19]: #fig:bio27


<figure><img src="bioinformatika32.png" width="40%" id="fig:bio27" /><figcaption style="text-align: center;"><b>Slika 3.19</b>: Matrica $Profile(Motifs)$ dobijena na osnovu nasumično izabranih $k$-grama</figcaption></figure>


Sa Slike [3.20] vidimo da  na osnovu profilne matrice možemo izračunati verovatnoću svakog $k$-grama u kolekciji sekvenci $Dna$. Iz svakog reda biramo najverovatniji $k$-gram i formiramo skup $k$-grama $Motifs$ i nastavljamo iteraciju. 


[3.20]: #fig:bio28

<figure><img src="bioinformatika33.png" width="50%" id="fig:bio28" /><figcaption style="text-align: center;"><b>Slika 3.20</b>:Verovatnoće svih $k$-grama iz $Dna$</figcaption></figure>



Kako jedno pokretanje algoritma nije dovoljno za dobijanje najboljeg rešenja, u praksi se ovaj algoritam pokreće hiljadama puta. Kao rezultat vraća skup motiva sa najnižim skorom. U Tabeli [3.4]  je prikazan rezultat ovog algoritma testiran na $benchmark$ kolekciji nakon $100$ $000$ pokretanja (svaki put sa novim izabranim $k$-gramima). Rezultat je kolekcija motiva $Motifs$ sa konsenzus niskom AAAAAAAAacaGGGG, pri čemu je $Score(Motifs)$ = 43, što znači da daje lošije rešenje od algoritma $Greedy$ $Motif$ $Laplace$ i algoritma $Median$ $String$, ali bolje rešenje od algoritma  $Greedy$ $Motif$.

[3.4]: #tab:tab5

<figure>
<figcaption style="text-align: center;"><b>Тabela 3.4</b>: Rezultati testiranja algoritama na $benchmark$ skupu</figcaption>
<table id="tab:tab5">
<thead>
<tr class="header">
<th style="text-align: center;">Algoritam</th>
<th style="text-align: center;">k</th>
<th style="text-align: center;">Rešenje</th>
<th style="text-align: center;">Skor</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td style="text-align: center;">$Median$ $String$</td>
<td style="text-align: center;">13</td>
<td style="text-align: center;">AAAAAtAGaGGGG</td>
<td style="text-align: center;">29</td>

</tr>
<tr class="even">
<td style="text-align: center;">$Median$ $String$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">Previše spor</td>
<td style="text-align: center;"></td>
</tr>
<tr class="even">
<td style="text-align: center;">$Greedy$ $Motif$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">gtAAAtAgaGatGtG</td>
<td style="text-align: center;">58</td>
</tr>
<tr class="even">
<td style="text-align: center;">$Greedy$ $Motif$ $Laplace$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">AAAAAtAgaGGGGtt</td>
<td style="text-align: center;">41</td>
</tr>
<tr class="even">
<td style="text-align: center;">Randomized $Motif$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">AAAAAAAAacaGGGG</td>
<td style="text-align: center;">43</td>
</tr>
<tr>
<td style="text-align: center;" colspan="2">Traženi $(k,d)$ motiv:</td> 
<td style="text-align: center;" colspan="1">AAAAAAAAGGGGGGG</td> 
<td style="text-align: center;"> </td> 
</tr>
</tbody>
</table>
</figure>

Ima smisla postaviti pitanje kako slučajno odabrani motivi mogu da nas dovedu do ovakvog rešenja. Verovatnoća da u kolekciji $Subtle$ $Motif$ $Search$ slučajnim izborom u jednoj sekvenci nađemo ubačeni motiv je $\frac{1}{(600-15+1)}$ = $\frac{1}{586}$, a verovatnoća da ćemo promašiti je $1-\frac{1}{586}$. Dakle, verovatnoća u celoj kolekciji je $(1-\frac{1}{586})^{10}$ $\sim$ 0.983, što znači da ukoliko algoritam pokrenemo 1000 puta, 983 puta ćemo promašiti i 17 puta pogoditi bar neki motiv.  Ako ne pogodimo motiv, pogodićemo deo niske generisan na slučajan način gde nam je svaki nukleotid generisan sa jednakom verovatnoćom na svakoj poziciji (0.25) i možemo očekivati uniformnu profilnu matricu (sve vrednosti 0.25) koja algoritam ne usmerava prema rešenju. Ukoliko pogodimo bar jedan motiv, vrednosti u matrici neće biti sasvim slučajne i matrica će usmeravati pretragu.


# Gibsovo uzorkovanje  <a id="par:gibsovosempliranje"></a>

Prethodno opisani algoritam $Randomized$ $Motif$ $Search$ podrazumeva promenu svih motiva iz $Motifs$ u jednoj iteraciji i tad ispravni motivi mogu biti odbačeni. Iz tog razloga modifikujemo algoritam tako da u svakoj iteraciji  bira jedan $k$-gram iz trenutnog skupa motiva $Motifs$, izbacuje ga, pravi profilnu matricu od ostatka motiva, a zatim ažurira matricu motiva samo nad izbačenim redom u matrici. Ova modifikacija algoritma $Randomized$ $Motif$ $Search$ se naziva Gibsovo uzorkovanje ($Gibbs$ $Motif$ $Search$).

U ovom algoritmu rezultat svakog koraka zavisi samo od rezultata prethodnog i  način odabira sledećeg koraka nije deterministički, već zasnovan na slučajnom uzorku, tj. koristi nasumično izabrane $k$-grame $Motifs$=($Motif_1$,...,$Motif_t$) da izabere bolji skup motiva, dok  $Randomized$ $Motif$ $Search$ deterministički bira nove motive koristeći
\[
Motifs( Profile ( Motifs ) , Dna ).
\]
 Gibsovo uzorkovanje bira nasumično jedan broj $i$ između $1$ i $t$, pri čemu je $t$ broj sekvenci iz skupa $Dna$ i menja $k$-gram iz $Motif_i$. Za opis rada ovog algoritma potreban je napredniji generator slučajnih brojeva. $Random(p_1,...,p_n)$ je defnisan za bilo koji skup nenegativnih brojeva koji nužno zadovoljava uslov $\sum_{i=1}^{n} p_i=1$.
Ukoliko je $\sum_{i=1}^{n} p_i= C>0$, onda $Random(p_1,...,p_n)=Random(p_1/C,...,p_n/C)$ pri čemu je $(p_1/C,...,p_n/C)$ raspodela verovatnoće. 

Potrebno je definisati šta predstavlja nasumično generisan  $k$-gram u niski $Text$. Za svaki $k$-gram $Pattern$ u  niski $Text$  računamo $Pr(Pattern|Text)$, što daje 
$n=|Text|-k+1$ verovatnoća $(p_1,...p_n)$. Možemo formirati generator slučajnih brojeva $Random(p_1,...,p_n)$ zasnovan na njima. Gibsovo uzorkovanje koristi ovaj generator slučajnih brojeva da izabere $k$-gram koji je nasumično generisan. Ova procedura se ponavlja $N$ puta, ali u praksi zavisi od raznih pravila zaustavljanja.


Ispod je data implementacija algoritma  $Gibbs$ $Motif$ $Search$.

In [31]:
import random
def most_probable_gibbs_k_mer(motif_profile, sequence, k):
    #u probabilities smeštamo verovatnoće k-grama iz sekvence sequence
    probabilities = []
    #u patterns smeštamo k-grame iz sekvence sequence
    patterns = []
    
    n = len(sequence)
    prob_sum = 0
    
    for i in range(n - k + 1):
        pattern = sequence[i : i + k]
        #za svaki k-gram iz sekvence racunamo verovatnoću na osnovu profilne matrice i dodajemo prob_sum
        prob = probability(motif_profile, pattern, k)
        prob_sum += prob
        #k-gram smestamo u patterns, a verovatnocu u probabilities
        patterns.append(pattern)
        probabilities.append(prob)
    #random poziciju dobijamo tako sto sumu verovatnoća pomnozimo sa nekim rendom brojem  
    random_pos = random.random() * prob_sum
    
    current_sum = 0
    #prolazimo redom kroz verovatnoće iz probabilities
    for i in range(n - k + 1):
        #uzimamo verovatnoću na i-toj poziciji i dodajemo sumi. Ukoliko suma postane veća ili jednaka od random_pos, kao rezultat vracamo patterns[i]
        prob = probabilities[i]
        current_sum += prob
        
        if current_sum >= random_pos:
            return patterns[i]

In [32]:
from copy import deepcopy
def gibbs_sampler(dna, k, t, N):
    motifs = []
    #prolazimo redom kroz sekvence, i biramo random poziciju odakle ce poceti motiv. Izabrane motive smestamo u niz motifs
    for sequence in dna:
        n = len(sequence)
        i = random.randrange(0, n - k + 1)
        motifs.append(sequence[i : i + k])
    #motive kopiramo u best_motifs  
    best_motifs = deepcopy(motifs)
    #skor motiva best_motifs smestamo u best_score
    best_score = get_score(best_motifs, k)
    
    #iteriramo N puta
    for _ in range(N):
        #izaberemo broj izmedju 0 i t
        i = random.randrange(0, t)
        #izbrisemo motiv na poziciji i u skupu motiva motifs
        del motifs[i]
        #racunamo profilnu matricu na osnovu preostalih motiva
        motif_profile = get_profile(motifs, k)
        #nalazimo najverovatniji k-gram i smestamo ga na i-tu poziciju u motifs
        motif_i = most_probable_gibbs_k_mer(motif_profile, dna[i], k)
        motifs.insert(i, motif_i)
        
        #racunamo trenutnu skor
        current_score = get_score(motifs, k)
        #ako je trenutni skor bolji od best_score, azuriramo ga
        if current_score < best_score:
            best_score = current_score
            best_motifs = deepcopy(motifs)
            
    return best_score, best_motifs

In [33]:
# Primer
dna = [
    'ttACCTtaac',
    'gATGTctgtc',
    'acgGCGTtag',
    'ccctaACGAg',
    'cgtcagAGGT'
]

k = 4
min_score = float('inf')
min_motifs = None
N = 100


for _ in range(100):
    best_score, best_motifs = gibbs_sampler(dna, k, len(dna), N)
    if best_score <= min_score:
        min_score = best_score
        min_motifs = best_motifs
        
print(min_motifs)

['Ttaa', 'Tctg', 'Ttag', 'ctaA', 'tcag']


Ilustrovaćemo primenu $Gibbs$ $Motifs$ $Search$ algoritma na primeru pronalaženja motiva unutar kolekcije sekvenci $Dna$ (Slika [3.21]). U početnom koraku iz svake sekvence su nasumično izabrani $k$-grami i nasumično je izabrana treća sekvenca za uklanjanje. 

[3.21]: #fig:bio29

<figure><img src="bioinformatika35.png" width="40%" id="fig:bio29" /><figcaption style="text-align: center;"><b>Slika 3.21</b>:Izbacivanje treće sekvence iz skupa sekvenci $Dna$</figcaption></figure>

Uklanjanjem treće sekvence, dobijamo novi skup motiva $Motifs$, a zatim računamo $Count(Motifs)$ i $Profile(Motifs)$ bez primene Laplasovog pravila (Slika [3.22]). 

[3.22]: #fig:bio30

<figure><img src="bioinformatika36.png" width="50%" id="fig:bio35" /><figcaption style="text-align: center;"><b>Slika 3.22</b>:Matrice bez primene Laplasovog pravila</figcaption></figure>

Sada koristeći $Profile(Motifs)$ računamo verovatnoće svih $4$-grama iz niske $ccgGCGTtag$. Na Slici [3.23] se uočava da su samo dve verovatnoće različite od nule. Susrećemo se sa problemom koji se javio i kod $Greedy$ $Motif$ $Search$. Problem ćemo rešiti primenom Laplasovog pravila.

[3.23]: #fig:bio31

<figure><img src="bioinformatika37.png" width="55%" id="fig:bio31" /><figcaption style="text-align: center;"><b>Slika 3.23</b>:Verovatnoće svih $k$-grama iz treće sekvence</figcaption></figure>


Kako Laplasovo pravilo sukcesije dodaje svakom elementu matrice Count(Motifs) 1, na Slici [3.24] se mogu videti ažurirane Count(Motifs) i Profile(Motifs). 

[3.24]: #fig:bio32

<figure><img src="bioinformatika38.png" width="50%" id="fig:bio32" /><figcaption style="text-align: center;"><b>Slika 3.24</b>:Matrice nakon primene Laplasovog pravila</figcaption></figure>

Nakon ažuriranja ove dve matrice, ponovo računamo verovatnoće svih $4$-grama iz niske $ccgGCGTtag$ (Slika [3.25]).

[3.25]: #fig:bio33


<figure><img src="bioinformatika39.png" width="55%" id="fig:bio33" /><figcaption style="text-align: center;"><b>Slika 3.25</b>:Verovatnoće svih $k$-grama iz treće sekvence</figcaption></figure>

Sekvencu $ccgGCGTtag$ vraćamo u kolekciju sekvenci $Dna$, ali ugrađeni motiv $ccgG$ iz treće sekvence skupa $Dna$ zamenjujemo sa $GCGT$. Sada biramo prvu sekvencu iz skupa $Dna$ za uklanjanje i ponavljamo postupak (Slika [3.26]). 


[3.26]: #fig:bio34

<figure><img src="bioinformatika40.png" width="40%" id="fig:bio34" /><figcaption style="text-align: center;"><b>Slika 3.26</b>:Uklanjanje prve sekvence iz skupa $Dna$ </figcaption></figure>

Na Slici [3.27] su  dat skup motiva $Motifs$ i $Profile(Motifs)$ bez primene Laplasovog pravila.


[3.27]: #fig:bio35

<figure><img src="bioinformatika41.png" width="50%" id="fig:bio35" /><figcaption style="text-align: center;"><b>Slika 3.27</b>:Matrice bez primene Laplasovog pravila  </figcaption></figure>

Potrebno je da ažuriramo $Count(Motifs)$ i $Profile(Motifs)$ koristeći Laplasovo pravilo (Slika [3.28]).

[3.28]: #fig:bio36

<figure><img src="bioinformatika42.png" width="50%" id="fig:bio36" /><figcaption style="text-align: center;"><b>Slika 3.28</b>:Matrice nakon primene Laplasovog pravila</figcaption></figure>


Sada računamo verovatnoće svih $4$-grama iz uklonjene prve sekvence skupa $Dna$ (Slika [3.29]).


[3.29]: #fig:bio37

<figure><img src="bioinformatika43.png" width="50%" id="fig:bio37" /><figcaption style="text-align: center;"><b>Slika 3.29</b>:Verovatnoće svih $4$-grama iz prve sekvence</figcaption></figure>

Vraćamo u skup sekvenci uklonjenu prvu sekvencu, ali umesto $taac$ ubacujemo  motiv $ACCT$. Zatim uklanjamo četvrtu sekvencu iz skupa $Dna$ i ponavljamo postupak (Slika [3.30]).

[3.30]: #fig:bio38

<figure><img src="bioinformatika44.png" width="40%" id="fig:bio38" /><figcaption style="text-align: center;"><b>Slika 3.30</b>:Uklanjanje četvrte sekvence iz skupa $Dna$</figcaption></figure>

Kao i u prethodnim postupcima, ažuriramo matrice $Count(Motifs)$ i $Profile(Motifs)$ koristeći Laplasovo pravilo (Slika [3.31]).

[3.31]: #fig:bio39

<figure><img src="bioinformatika45.png" width="50%" id="fig:bio39" /><figcaption style="text-align: center;"><b>Slika 3.31</b>:Matrice nakon primene Laplasovog pravila </figcaption></figure>


Verovatnoće svih $4$-grama iz uklonjenje četvrte sekvence su prikazane na Slici [3.32].

[3.32]: #fig:bio40


<figure><img src="bioinformatika46.png" width="50%" id="fig:bio40" /><figcaption style="text-align: center;"><b>Slika 3.32</b>:Verovatnoće svih $4$-grama iz četvrte sekvence </figcaption></figure>

Sada vraćamo uklonjenu četvrtu sekvencu, ali motiv $acta$ menjamo sa $ACGA$. Algoritam počinje da konvergira.

Opisani algoritam ne mora uvek da vrati globalni maksimum. On mora biti pokrenut više puta kako bismo bili sigurni da je pronašao globalni maksimum, a ne lokalni maksimum. U opštem slučaju ovaj algoritam brzo konvergira i mala je verovatnoća da se će se zaglaviti u lokalnom maksimumu. U Tabeli [3.5] je prikazan rezultat algoritma testiran na $benchmark$ kolekciji. $Gibbs$ $Motif$ $Search$ pronalazi kolekciju motiva $Motifs$ sa konsenzus  niskom AAAAAAgAGGGGGGt, pri čemu je $Score(Motifs)$ = 38.

[3.5]: #tab:tab6

<figure>
<figcaption style="text-align: center;"><b>Тabela 3.5</b>: Rezultati testiranja algoritama na $benchmark$ skupu</figcaption>
<table id="tab:tab6">
<thead>
<tr class="header">
<th style="text-align: center;">Algoritam</th>
<th style="text-align: center;">k</th>
<th style="text-align: center;">Rešenje</th>
<th style="text-align: center;">Skor</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td style="text-align: center;">$Median$ $String$</td>
<td style="text-align: center;">13</td>
<td style="text-align: center;">AAAAAtAGaGGGG</td>
<td style="text-align: center;">29</td>

</tr>
<tr class="even">
<td style="text-align: center;">$Median$ $String$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">Previše spor</td>
<td style="text-align: center;"></td>
</tr>
<tr class="even">
<td style="text-align: center;">$Greedy$ $Motif$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">gtAAAtAgaGatGtG</td>
<td style="text-align: center;">58</td>
</tr>
<tr class="even">
<td style="text-align: center;">$Greedy$ $Motif$ $Laplace$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">AAAAAtAgaGGGGtt</td>
<td style="text-align: center;">41</td>
</tr>
<tr class="even">
<td style="text-align: center;">Randomized $Motif$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">AAAAAAAAacaGGGG</td>
<td style="text-align: center;">43</td>
</tr>
<tr class="even">
<td style="text-align: center;">$Gibbs$ $Motif$</td>
<td style="text-align: center;">15</td>
<td style="text-align: center;">AAAAAAgAGGGGGGt</td>
<td style="text-align: center;">38</td>
</tr>

<tr>
<td style="text-align: center;" colspan="2">Traženi $(k,d)$ motiv:</td>  
<td style="text-align: center;">AAAAAAAAGGGGGGG</td>
 <td style="text-align: center;"></td>   
</tr>
</tbody>
</table>
</figure>

# EM algoritam <a id="par:emalgoritam"></a>

U potrazi za regulatornim motivima u skupu sekvenci $Dna$, tj. pozicijama gde tražene instance motiva počinju u svakoj sekvenci analiziraćemo još jedan probabilistički algoritam. Na Slici [3.33] je dat primer početnih pozicija traženih instanci motiva. Za datu kolekciju sekvenci, neka je $Z$ matrica u kojoj $Z_{ij}$ odgovara verovatnoći da instanca motiva počinje na poziciji $j$ u sekvenci $i$, a PWM ($probability$ $weight$ $matrix$) matrica koja sadrži frekvenciju svakog nukleotida na svakoj poziciji u motivu i u ostatku sekvence. Tada govorimo o raspodeli nukleotida unutar motiva (motivska raspodela (eng. $motif$ $distribution$)) i o raspodeli nukleotida izvan motiva (pozadinska raspodela (eng. $background$ $distribution$)). Možemo koristiti matricu PWM za dobijanje matrice $Z$, a zatim koristeći matricu $Z$ da ažuriramo matricu PWM. Ova dva koraka se ponavljaju određeni broj puta ili do konvergencije. Algoritam koji se zasniva na ova dva koraka naziva se $EM$ algoritam ($Expectation-Maximization$) i u nastavku će biti detaljno opisan njegov rad.





[3.33]: #fig:bio41


<figure><img src="em.png" width="35%" id="fig:bio42" /><figcaption style="text-align: center;"><b>Slika 3.33</b>:Primer početnih pozicija traženih motiva</figcaption></figure>


# E korak

Prvi korak u EM algoritmu je generisanje matrice PWM. Na početku PWM možemo inicijalizovati nasumično birajući početne pozicije motiva. Ukoliko tu matricu označimo sa $p$, $p_{c,k}$ je verovatnoća da se nukleotid $c$ pojavi na poziciji $k$ u motivu. Takođe računamo $p_{c,k}$ za $k=0$ što predstavlja pozadinsku raspodelu motiva, tj. verovatnoću nukleotida $c$ na pozicijama u sekvenci izvan (pre i posle) kandidata  motiva.


U E koraku generišemo matricu Z, koja nam predstavlja verovatnoće početnih pozicija instanci motiva u svakoj sekvenci. Neka je dat skup sekvenci $Dna$, pri čemu je $X_{i}$  $i$-ta sekvenca iz $Dna$ i neka je $L$  dužina sekvence, a $W$ dužina motiva.
Tada verovatnoću sekvence $X_{i}$  pri uslovu da motiv u toj sekvenci počinje na poziciji $j$ dobijamo formulom:
$$
Pr^t(X_{i} | Z_{ij} =1, p) = Pr(X_{i} | Z_{ij} =1) = \prod\limits_{k=1}^{j-1} p_{c_{k},0} \prod\limits_{k=j}^{j+W-1} p_{c_{k},k-j+1} \prod\limits_{k=j+W}^{L} p_{c_{k},0}.
$$

Prvi i poslednji proizvod u formuli iznad odgovaraju verovatnoćama da deo sekvence koji nije kandidat da bude motiv potiče iz neke pozadinske raspodele, dok srednji proizvod predstavlja verovatnoću da razmatrani kandidat dolazi iz motivske raspodele motiva.

$Z_{ij}$ u iteraciji $t$ računamo po sledećoj formuli:

$Z_{ij}^{t}$ = $\frac{Pr^t(X_{i} | Z_{ij} = 1) Pr^t(Z_{ij}=1)} {\sum\limits_{k=1}^{L-W+1} Pr^t(X_{i} | Z_{ik} = 1) Pr^t(Z_{ik}=1)} $.



U nastavku ćemo kroz primer prikazati $E$ korak. Neka je data $i$-ta sekvenca $GCTGTAG$, matrica $p$, gde su nasumično izabrane vrednosti, pri čemu je $W=3$ (Slika [3.34]).

[3.34]: #fig:bio43



<figure><img src="p.png" width="35%" id="fig:bio43" /><figcaption style="text-align: center;"><b>Slika 3.34</b>: Primer PWM matrice</figcaption></figure>


Na osnovu matrice $p$, dobijamo da je:

-  $Z_{i,1}$ = $0.3$ $\cdot$ $0.2$ $\cdot$ $0.1$ $\cdot$ $0.25$ $\cdot$ $0.25$ $\cdot$ $0.25$ $\cdot$ $0.25$
-  $Z_{i,2}$ = $0.25$ $\cdot$ $0.4$ $\cdot$ $0.2$ $\cdot$ $0.6$ $\cdot$ $0.25$ $\cdot$ $0.25$ $\cdot$ $0.25$
-  $Z_{i,3}$ = $0.25$ $\cdot$ $0.25$ $\cdot$ $0.2$ $\cdot$ $0.1$ $\cdot$ $0.1$ $\cdot$ $0.25$ $\cdot$ $0.25$
-  $Z_{i,4}$ = $0.25$ $\cdot$ $0.25$ $\cdot$ $0.25$ $\cdot$ $0.3$ $\cdot$ $0.2$ $\cdot$ $0.2$ $\cdot$ $0.25$
-  $Z_{i,5}$ = $0.25$ $\cdot$ $0.25$ $\cdot$ $0.25$ $\cdot$ $0.25$ $\cdot$ $0.2$ $\cdot$ $0.5$ $\cdot$ $0.6$.


# M korak

Nakon što smo izračunali Z, možemo koristiti rezultat kako bismo ažurirali PWM i verovatnoće nukleotida na pozicijama u sekvenci izvan (pre i posle) kandidata motiva. Ukoliko sa $n_{c}$  označimo ukupan broj pojavljivanja karaktera $c$ u skupu sekvenci, tada $n_{c,k}$ definišemo kao:
$$
n_{c,k}=\begin{cases}
			\sum\limits_{i}^{} \sum\limits_{\{j | X_{i,j+k-1} = c\}}^{}Z_{ij}, & \text{$k$ > 0}\\
            n_{c} - \sum\limits_{j=1}^{W}n_{c,j}, & \text{$k$ = 0},
		 \end{cases}
$$ 
pri čemu unutrašnja suma za $k>0$ ide po $j$-tim pozicijama koja može biti kanditat za početnu poziciju motiva dužine $W$ i na kojima se nalazi nukleotid $c$.



PWM možemo ažurirati u iteraciji $t+1$ koristeći sledeću formulu:
$$
p^{t+1}_{c,k} = \frac{n_{c,k}+d_{c,k}}{\sum\limits_{b} (n_{b,k}+d_{b,k})},
$$
pri čemu su $d_{c,k}$ i $d_{b,k}$  pseudovrednosti (najčešće $1$), a $b$ $\in$ $\{A,C,G,T\}$.


U nastavku ćemo kroz primer prikazati $M$ korak. Neka je na Slici [3.35] dat skup sekvenci i matrica $Z$.

[3.35]: #fig:bio44


<figure><img src="emprimer.png" width="35%" id="fig:bio44" /><figcaption style="text-align: center;"><b>Slika 3.35</b>: Kolekcija sekvenci $Dna$ i primer matrice $Z$</figcaption></figure>


$p_{A,1}$ računamo po formuli:
$$
p_{A,1} = \frac{Z_{1,1} + Z_{1,3} + Z_{2,1} + Z_{3,3}+ 1}{Z_{1,1} + Z_{1,2} ... + Z_{3,3} + Z_{3,4} + 4}.
$$

Ispod je data implementacija $EM$ algoritma.

In [51]:
import numpy as np

#ulazni parametri: skup sekvenci, dužina traženih motiva i broj iteracija EM algoritma                
def em_algoritam (sequences, motif_length, num_iterations):
    #u promenjivu num_sequences smeštamo broj sekvenci
    num_sequences = len(sequences) 
    # u promenjivu sequence_length smeštamo dužinu jedne sekvence
    sequence_length = len(sequences[0]) 
    
    # Inicijalizacija parametara
    background_probs = np.ones(4) / 4  #jednaka verovatnoća za svaki nukelotid [0.25,0.25,0.25.0.25]
    motif_probs = np.random.rand(motif_length, 4)  # slučajne verovatnoće za motiv
    
    
  # E-korak i M-korak se ponavljaju sve dok se ne dostigne maksimalan broj iteracija
    for iteration in range(num_iterations):
        # E-korak
        expected_hidden_vars = np.zeros((num_sequences, sequence_length - motif_length + 1)) 
        
    #za svaku sekvencu se racuna ocekivana vrednost skrivenih variabli, koja predstavlja verovatnocu da svaka pozicija u sekvenci pripada motivu
        for i in range(num_sequences):
    #u promenjivu sequence smeštamo i-tu sekvencu
            sequence = sequences[i]
            
            for j in range(sequence_length - motif_length + 1):
    #uzimamo sve moguće podsekvence dužine motif_length u i-oj sekvenci
                subsequence = sequence[j:j+motif_length]

                motif_prob = np.prod([motif_probs[k, base_to_index(subsequence[k])] for k in range(motif_length)])
                
                background_prob = np.prod([background_probs[base_to_index(sequence[m])] for m in range(sequence_length) if m < j or m >= j+motif_length])

    #ocekivana vrednost se racuna kao prosek verovatnoce motiva i verovatnoce pozadine za svaku poziciju u sekvenci.
                expected_hidden_vars[i, j] = (motif_prob * background_prob)
            for k in range(sequence_length - motif_length + 1):
                  expected_hidden_vars[i, k]  =  expected_hidden_vars[i, k]   /   np.sum([expected_hidden_vars[i, m] for m in range(sequence_length - motif_length + 1)])
        # M-korak koristeći očekivane vriednosti, ažuriraju se verovatnoce motiva i pozadine.
    #matrice motif_probs i background_probs postavimo na 0 matrice
        motif_probs = np.zeros((motif_length, 4))
        background_probs = np.zeros(4)
        
        for i in range(num_sequences):
            sequence = sequences[i]
            
            for j in range(sequence_length - motif_length + 1):
                subsequence = sequence[j:j+motif_length]
    #verovatnoce motiva se računaju kao prosek očekivanih vrednosti motiva preko svih sekvenci
            for k in range(motif_length):
                        if sequence[j+k] == subsequence[k]:
                            motif_probs[k, base_to_index(subsequence[k])] += expected_hidden_vars[i, j] 
      #verovatnoce pozadine se računaju kao prosek očekivanih vrednosti pozadine preko svih sekvenci.

            for k in range(sequence_length):
                    if k < j or k >= j+motif_length:
                        background_probs[base_to_index(sequence[k])] += numcharacter(sequences,sequence[k]) - np.sum([motif_probs[m,base_to_index(subsequence[m])] for m in range(motif_length) if sequence[m] == sequence[k]])
            motif_probs = (motif_probs + 1) / (np.sum(motif_probs, axis=1, keepdims=True) + 4)
            background_probs = (background_probs + 1) /  (np.sum(background_probs) + 4)
        
    #algoritam vraca verovatnocu pozadine i verovatnocu motiva
    return background_probs, motif_probs



In [52]:
#funkcija koja na osnovu nukelotida vraca indeks
def base_to_index(base):
    if base == 'A':
        return 0
    elif base == 'C':
        return 1
    elif base == 'G':
        return 2
    elif base == 'T':
        return 3

In [53]:
def numcharacter(sequences,char):
    num = 0
    num_sequences = len(sequences)
    for i in range(num_sequences):
        sequence = sequences[i]
        for j in range(len(sequence)):
            if sequence[j] == char:
                num += 1
    return num
            

In [54]:
def index_to_base(index):
    if index == 0:
        return 'A'
    elif index == 1:
        return 'C'
    elif index == 2:
        return 'G'
    elif index == 3:
        return T

In [55]:
# Primer korištenj
sequences = ['ACGTAGC', 'CGTAGCT', 'GCTAGCG']
motif_length = 3
num_iterations = 15

background_probs, motif_probs = em_algoritam(sequences, motif_length, num_iterations)

print("Procenjene verovatnoce pozadine:")
print(background_probs)
print("Procenjene verovatnoce motiva:")
print(motif_probs)


motif = ''.join([index_to_base(np.argmax(motif_probs[i])) for i in range(motif_length)])
print("Pronađeni motiv:")
print(motif)



Procenjene verovatnoce pozadine:
[0.20618412 0.27835006 0.31958689 0.19587892]
Procenjene verovatnoce motiva:
[[0.24998414 0.24998086 0.25005415 0.24998086]
 [0.24998086 0.25005415 0.24998414 0.24998086]
 [0.24998086 0.24998414 0.25004155 0.24999346]]
Pronađeni motiv:
GCG


E i M korake ponavljamo unapred određeni broj puta  ili sve do konvergencije. Jedan od načina da znamo da li je profilna matrica konvergirala jeste da pratimo koliko se svaki element u PWM promeni nakon maksimiziranja. Algoritam se zaustavlja ukoliko je promena ispod odabranog praga. Preporučljivo je ponovno pokrenuti algoritam sa različitim početnim pozicijama kako bi se pokušala smanjiti mogućnost konvergiranja na lokalnom maksimumu.

Tabela [3.5] neće biti proširena rezultatima testiranja $EM$ algoritma na $benchmark$ kolekciji, budući da su podaci iz tabele preuzeti iz udžbenika u kome ovaj algoritam nije obrađen, a u kome je prikazan samo segment kolekcije sekvenci. 

[3.5]: #tab:tab6